In [227]:
import glob, json, pathlib
from tqdm.notebook import tqdm
sanitychecks = list(glob.glob(r"D:\ig_pipeline\cad\*\*\artifacts\sanitycheck.json"))
jsons = {}
for x in tqdm(sanitychecks):
    with open(x, "r") as f:
        name = "/".join(pathlib.Path(x).parts[-4:-2])
        jsons[name] = json.load(f)

  0%|          | 0/91 [00:00<?, ?it/s]

In [228]:
total = len(jsons)
failed = {name for name, x in jsons.items() if x["errors"]}
total, len(failed)

(91, 90)

In [266]:
from collections import defaultdict
import re
error_templates_and_ignore = {
    "Category (.*) for object .* does not exist on spreadsheet.": True,
    "(.*) has too many vertices: .* > .*": True,
    "(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.": False,
    "(.*) is missing in instance .*, so relative transform check cannot be completed.": False,
    "(.*) has bad name.": False,
    "(.*) has negative object offset scale.*": False,
    "(.*) has scale that is not 1. Reset scale.": False,
    ".*No instance ID 0 instance of (.*)": False,
    "Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.": False,
    "All instances of (.*) do not have contiguous instance IDs. Missing: .*": False,
    "(.*) has disallowed type.*": False,
    "Light object (.*) should not have object offset rotation. Reset pivot.": False,
    "(.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.": False,
    "(.*) has different pivot offset rotation \(by (.*)\). Match pivots on each instance.": False,
    "(.*) has different shear. Match scaling axes on each instance.": False,
    "Model ID (.*) contains 'todo'.": False,
    "(.*) should not have an upper side.": False,
    "(.*) should have an upper side.": False,
    "(.*) has different vertex count than recorded in provider.*": False,
    "(.*) has different face count than recorded in provider.*": False,
    "(.*) has no collision mesh. Create a collision mesh.": False,
    "(.*) has different UV unwrapping than recorded. Reunwrap the object.": False,
    "Inconsistent link sets within model (.*)": False,
    "(.*) is missing meta link: fillable.": False,
    "(.*) is missing meta link: heatsource.": False,
    "(.*) is missing meta link: togglebutton.": False,
    "(.*) is missing meta link: particleapplier.": False,
    "(.*) is missing meta link: fluidsource.": False,
    "(.*) is missing meta link: fluidsink.": False,
    "(.*) is missing meta link: togglebutton.": False,
    "(.*) element .* has too many vertices .*": True,
    "Model ID (.*) requires joints but has no joints.": False,
    "Expected meta links for (.*) are missing: .*": False,
    "Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*": False,
    "Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements.": True,
    "Cannot validate clothness: category (.*) not found in taxonomy.": True,
    "(.*) element .* is not a volume": True,
    "(.*) element .* has elements trimesh still finds splittable": True,
    "Object (.*) has .* fillable meshes. Should have no more than one.": False,
    "Model (.*) has category .* that is neither the from or to element in the rename file.": True,
    "(.*) has meta link not required by its synset: .*": True,
    "(.*) meta type .* ID .* has non-continuous subids .*": False,
    "Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.": True,
    "(.*) is in the deletion queue. Delete the object.": False,
}
error_templates = list(error_templates_and_ignore.keys())
error_matches = defaultdict(set)
error_matches_by_file = defaultdict(lambda: defaultdict(set))
unknown_error_files = []
for name, f in jsons.items():
    unknown_error = False
    for error in f["errors"] + f["warnings"]:
        for i, tmpl in enumerate(error_templates):
            match = re.fullmatch(tmpl, error, re.S)
            if match is not None:
                if not error_templates_and_ignore[tmpl]:
                    error_matches[tmpl].add(match)
                    error_matches_by_file[tmpl][name].add(match)

                break
        else:
            print(error)
            unknown_error = True
    if unknown_error:
        unknown_error_files.append(name)
        
print("\nCounts")
for tmpl, objs in error_matches.items():
    print(f"{tmpl}: {len(objs)}")

print("\n\nCounts by file")
for tmpl, matches_by_file in error_matches_by_file.items():
    all_matches = error_matches[tmpl]
    if not all_matches:
        continue
    print(f"\n{tmpl}")
    for name, matches in sorted(matches_by_file.items(), key=lambda x: x[1], reverse=True):
        if not matches:
            break
        print(f"  {name}: {len(matches)}")


Counts
(.*) is missing meta link: fillable.: 918
Model ID (.*) requires joints but has no joints.: 126
(.*) is missing meta link: particleapplier.: 4
Model ID (.*) contains 'todo'.: 7
(.*) is missing meta link: togglebutton.: 201
Object (.*) has .* fillable meshes. Should have no more than one.: 2
(.*) meta type .* ID .* has non-continuous subids .*: 1
(.*) is in the deletion queue. Delete the object.: 255
(.*) has no collision mesh. Create a collision mesh.: 14
(.*) is missing meta link: heatsource.: 22
(.*) is missing meta link: fluidsink.: 36
(.*) is missing meta link: fluidsource.: 36
Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 139
(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 121
(.*) has different shear. Match scaling axes on each instance.: 2
(.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 84
(.*) has different

In [264]:
from IPython.display import display, Markdown

# Print the number of kinds of error for each target
output = []
output.append("\n\n# Counts by error")
for name in jsons.keys():
  all_errors = {}
  for tmpl, matches_by_file in error_matches_by_file.items():
      if error_templates_and_ignore[tmpl]:
          continue
      matches = matches_by_file[name]
      if not matches:
          continue
      all_errors[tmpl] = matches

  if all_errors:
      output.append(f"\n## {name}")
      for k, v in sorted(all_errors.items(), key=lambda x: len(x[1]), reverse=True):
          output.append(f"  ### {k}: {len(v)}")
          for match in v:
            output.append("    " + match.group(0))
          output.append("")

display(Markdown("\n".join(output)))



# Counts by error

## objects/batch-00
  ### (.*) is missing meta link: fillable.: 10
    aspeds is missing meta link: fillable.
    ntedfx is missing meta link: fillable.
    dkizyc is missing meta link: fillable.
    amhlqh is missing meta link: fillable.
    heqtlk is missing meta link: fillable.
    bnkjle is missing meta link: fillable.
    hmzafz is missing meta link: fillable.
    jpvcjv is missing meta link: fillable.
    petcxr is missing meta link: fillable.
    gewjwh is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID gewjwh requires joints but has no joints.

  ### (.*) is missing meta link: particleapplier.: 1
    izwycf is missing meta link: particleapplier.

  ### Model ID (.*) contains 'todo'.: 1
    Model ID jtodol contains 'todo'.


## objects/batch-01
  ### (.*) is missing meta link: fillable.: 11
    deudkt is missing meta link: fillable.
    msaevo is missing meta link: fillable.
    wopjex is missing meta link: fillable.
    nbuspz is missing meta link: fillable.
    mgbeah is missing meta link: fillable.
    adxzhe is missing meta link: fillable.
    strtoe is missing meta link: fillable.
    xifive is missing meta link: fillable.
    lageli is missing meta link: fillable.
    qbxfmv is missing meta link: fillable.
    gpnsij is missing meta link: fillable.


## objects/batch-02
  ### (.*) is missing meta link: fillable.: 9
    buoocq is missing meta link: fillable.
    kkmkbd is missing meta link: fillable.
    spopfj is missing meta link: fillable.
    ftzmow is missing meta link: fillable.
    qpzbqp is missing meta link: fillable.
    fhdyrj is missing meta link: fillable.
    wkxtxh is missing meta link: fillable.
    nhzrei is missing meta link: fillable.
    cpozxi is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID otyngn requires joints but has no joints.


## objects/batch-03
  ### (.*) is missing meta link: fillable.: 11
    jblalf is missing meta link: fillable.
    pdzaca is missing meta link: fillable.
    zpddxu is missing meta link: fillable.
    fedafr is missing meta link: fillable.
    tvtive is missing meta link: fillable.
    vcwsbm is missing meta link: fillable.
    yjmnej is missing meta link: fillable.
    qdnmwg is missing meta link: fillable.
    kkjiko is missing meta link: fillable.
    wvhmww is missing meta link: fillable.
    pkkgzc is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID zvidlq requires joints but has no joints.
    Model ID qdnmwg requires joints but has no joints.

  ### (.*) is missing meta link: togglebutton.: 1
    anslwz is missing meta link: togglebutton.


## objects/batch-04
  ### (.*) is missing meta link: fillable.: 8
    hacehh is missing meta link: fillable.
    qlxhhh is missing meta link: fillable.
    gklybu is missing meta link: fillable.
    qhnpmc is missing meta link: fillable.
    oshwps is missing meta link: fillable.
    cvdbum is missing meta link: fillable.
    wtepsx is missing meta link: fillable.
    jwxbpa is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID bqpmsv requires joints but has no joints.
    Model ID wcqjew requires joints but has no joints.

  ### Object (.*) has .* fillable meshes. Should have no more than one.: 1
    Object L-ink_bottle-gcyvrx-0-Tglass has 2 fillable meshes. Should have no more than one.

  ### (.*) meta type .* ID .* has non-continuous subids .*: 1
    L-ink_bottle-gcyvrx-0 meta type fillable ID 0 has non-continuous subids ['0', '0']


## objects/batch-05
  ### (.*) is missing meta link: fillable.: 10
    pnkiag is missing meta link: fillable.
    svhjxd is missing meta link: fillable.
    vhglly is missing meta link: fillable.
    incirm is missing meta link: fillable.
    gjrero is missing meta link: fillable.
    bfaqfe is missing meta link: fillable.
    svkdji is missing meta link: fillable.
    wgcgia is missing meta link: fillable.
    yvhmex is missing meta link: fillable.
    qatgeb is missing meta link: fillable.


## objects/batch-06
  ### (.*) is missing meta link: fillable.: 2
    hjxczh is missing meta link: fillable.
    hvaucj is missing meta link: fillable.

  ### Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.: 1
    Cannot validate meta links and joints for model ID lujosc: Category haystack not found in taxonomy.

  ### Cannot validate clothness: category (.*) not found in taxonomy.: 1
    Cannot validate clothness: category haystack not found in taxonomy.

  ### (.*) is in the deletion queue. Delete the object.: 1
    L-haystack-lujosc-0 is in the deletion queue. Delete the object.


## objects/batch-07
  ### (.*) is missing meta link: fillable.: 7
    ctrngh is missing meta link: fillable.
    jdddsr is missing meta link: fillable.
    zdvgol is missing meta link: fillable.
    ivbrtz is missing meta link: fillable.
    hkwtnf is missing meta link: fillable.
    pjinwe is missing meta link: fillable.
    oxivmf is missing meta link: fillable.

  ### (.*) is in the deletion queue. Delete the object.: 7
    potato-teqeai-3 is in the deletion queue. Delete the object.
    potato-teqeai-2 is in the deletion queue. Delete the object.
    potato-teqeai-4 is in the deletion queue. Delete the object.
    potato-teqeai-5 is in the deletion queue. Delete the object.
    potato-teqeai-1 is in the deletion queue. Delete the object.
    potato-teqeai-6 is in the deletion queue. Delete the object.
    potato-teqeai-0 is in the deletion queue. Delete the object.

  ### (.*) is missing meta link: particleapplier.: 1
    rwmotb is missing meta link: particleapplier.


## objects/batch-08
  ### (.*) is missing meta link: fillable.: 9
    kasebx is missing meta link: fillable.
    xjzyfc is missing meta link: fillable.
    hzspwg is missing meta link: fillable.
    wryghu is missing meta link: fillable.
    yhurut is missing meta link: fillable.
    hliauj is missing meta link: fillable.
    gewlsk is missing meta link: fillable.
    lpanoc is missing meta link: fillable.
    bnored is missing meta link: fillable.

  ### (.*) has no collision mesh. Create a collision mesh.: 3
    lily-jfeimu-0 has no collision mesh. Create a collision mesh.
    lily-jtouad-0 has no collision mesh. Create a collision mesh.
    lily-jobzwo-0 has no collision mesh. Create a collision mesh.


## objects/batch-09
  ### (.*) is in the deletion queue. Delete the object.: 4
    L-vase-dxnzuk-0 is in the deletion queue. Delete the object.
    sliced_lemon-gcjjjt-0 is in the deletion queue. Delete the object.
    sliced_lemon-andair-0 is in the deletion queue. Delete the object.
    sliced_lemon-uvncuu-0 is in the deletion queue. Delete the object.

  ### (.*) is missing meta link: fillable.: 3
    igqaol is missing meta link: fillable.
    dhkkfo is missing meta link: fillable.
    dxnzuk is missing meta link: fillable.


## objects/batch-10
  ### (.*) is missing meta link: fillable.: 6
    fvkdos is missing meta link: fillable.
    uvhflb is missing meta link: fillable.
    qthouj is missing meta link: fillable.
    ahtzhp is missing meta link: fillable.
    oqiwhb is missing meta link: fillable.
    kvithq is missing meta link: fillable.


## objects/batch-11
  ### (.*) is missing meta link: fillable.: 6
    glwebh is missing meta link: fillable.
    jnjtrl is missing meta link: fillable.
    bktljr is missing meta link: fillable.
    ivuveo is missing meta link: fillable.
    nawrfs is missing meta link: fillable.
    ruryqd is missing meta link: fillable.


## objects/batch-12
  ### (.*) is missing meta link: fillable.: 7
    oadvet is missing meta link: fillable.
    ogkdoh is missing meta link: fillable.
    xtqbuf is missing meta link: fillable.
    nsxhvs is missing meta link: fillable.
    wqgndf is missing meta link: fillable.
    pihjqa is missing meta link: fillable.
    pxvcyu is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID gejwoi requires joints but has no joints.

  ### (.*) is in the deletion queue. Delete the object.: 1
    cleansing_bottle-nsxhvs-0 is in the deletion queue. Delete the object.


## objects/batch-13
  ### (.*) is missing meta link: fillable.: 3
    coanpv is missing meta link: fillable.
    nodcpg is missing meta link: fillable.
    nhodax is missing meta link: fillable.


## objects/custom-aa
  ### (.*) is missing meta link: fillable.: 3
    ycbbwl is missing meta link: fillable.
    brimns is missing meta link: fillable.
    songes is missing meta link: fillable.


## objects/legacy_batch-00
  ### (.*) is missing meta link: fillable.: 60
    gvtucm is missing meta link: fillable.
    kdbgpm is missing meta link: fillable.
    qacthv is missing meta link: fillable.
    rvpunw is missing meta link: fillable.
    yohygj is missing meta link: fillable.
    jhymlr is missing meta link: fillable.
    pzjecb is missing meta link: fillable.
    qkgfln is missing meta link: fillable.
    ybntlp is missing meta link: fillable.
    ilofmb is missing meta link: fillable.
    pllcur is missing meta link: fillable.
    xpdbgf is missing meta link: fillable.
    hrdeys is missing meta link: fillable.
    plccav is missing meta link: fillable.
    rntwkg is missing meta link: fillable.
    xiurwn is missing meta link: fillable.
    jrhgeu is missing meta link: fillable.
    pkdnbu is missing meta link: fillable.
    wesxdp is missing meta link: fillable.
    yxfueo is missing meta link: fillable.
    glefdh is missing meta link: fillable.
    imyaje is missing meta link: fillable.
    phoesw is missing meta link: fillable.
    vespxk is missing meta link: fillable.
    getjzo is missing meta link: fillable.
    immwzb is missing meta link: fillable.
    olgoza is missing meta link: fillable.
    vceupd is missing meta link: fillable.
    kubcdk is missing meta link: fillable.
    bamfsz is missing meta link: fillable.
    nyfebf is missing meta link: fillable.
    ojceew is missing meta link: fillable.
    ujniob is missing meta link: fillable.
    gcrjan is missing meta link: fillable.
    hnxyev is missing meta link: fillable.
    nrlayx is missing meta link: fillable.
    rvunhj is missing meta link: fillable.
    twhums is missing meta link: fillable.
    kenkcw is missing meta link: fillable.
    nnvyol is missing meta link: fillable.
    ttmejh is missing meta link: fillable.
    nddvba is missing meta link: fillable.
    tolupn is missing meta link: fillable.
    mxrzpj is missing meta link: fillable.
    ncbfwr is missing meta link: fillable.
    sxppcj is missing meta link: fillable.
    eupgpt is missing meta link: fillable.
    mbmbpa is missing meta link: fillable.
    mwgceh is missing meta link: fillable.
    smujam is missing meta link: fillable.
    dajebq is missing meta link: fillable.
    lhucjo is missing meta link: fillable.
    lwjdmj is missing meta link: fillable.
    slgzfc is missing meta link: fillable.
    cjcyed is missing meta link: fillable.
    kxushk is missing meta link: fillable.
    leizjb is missing meta link: fillable.
    ryznya is missing meta link: fillable.
    zuwvdo is missing meta link: fillable.
    bycegi is missing meta link: fillable.

  ### (.*) has no collision mesh. Create a collision mesh.: 2
    bottom_cabinet-xpdbgf-0-base_link has no collision mesh. Create a collision mesh.
    bottom_cabinet-kxushk-0-base_link has no collision mesh. Create a collision mesh.


## objects/legacy_batch-01
  ### (.*) is missing meta link: fillable.: 6
    gulceo is missing meta link: fillable.
    fdjykf is missing meta link: fillable.
    oobbwn is missing meta link: fillable.
    jfhrwj is missing meta link: fillable.
    enpxbn is missing meta link: fillable.
    azdgtl is missing meta link: fillable.


## objects/legacy_batch-02
  ### (.*) has no collision mesh. Create a collision mesh.: 1
    breakfast_table-pllvrt-0-base_link has no collision mesh. Create a collision mesh.


## objects/legacy_batch-03
  ### (.*) is missing meta link: fillable.: 10
    vzwhbg is missing meta link: fillable.
    znfvmj is missing meta link: fillable.
    cgugie is missing meta link: fillable.
    gtnmis is missing meta link: fillable.
    ynrhuk is missing meta link: fillable.
    dugpby is missing meta link: fillable.
    tnlzww is missing meta link: fillable.
    xlmier is missing meta link: fillable.
    wvgsqk is missing meta link: fillable.
    dngvvi is missing meta link: fillable.

  ### (.*) has no collision mesh. Create a collision mesh.: 2
    dishwasher-tnlzww-0-base_link has no collision mesh. Create a collision mesh.
    nightstand-gjwjtj-0-Textrapart has no collision mesh. Create a collision mesh.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID qzwijs requires joints but has no joints.


## objects/legacy_batch-04
  ### (.*) is missing meta link: fillable.: 29
    nkrgez is missing meta link: fillable.
    spojpj is missing meta link: fillable.
    motill is missing meta link: fillable.
    ozzcvs is missing meta link: fillable.
    lsyzkh is missing meta link: fillable.
    oztrra is missing meta link: fillable.
    jvdbxh is missing meta link: fillable.
    yegcau is missing meta link: fillable.
    fqitbw is missing meta link: fillable.
    wamrju is missing meta link: fillable.
    fqhdne is missing meta link: fillable.
    qiausz is missing meta link: fillable.
    fmelap is missing meta link: fillable.
    tactqn is missing meta link: fillable.
    exgntg is missing meta link: fillable.
    ufhpbn is missing meta link: fillable.
    eobsmt is missing meta link: fillable.
    twjqua is missing meta link: fillable.
    dmwxyl is missing meta link: fillable.
    qohxjq is missing meta link: fillable.
    zjtaus is missing meta link: fillable.
    pluwfl is missing meta link: fillable.
    dhguym is missing meta link: fillable.
    vdedzt is missing meta link: fillable.
    bmsclc is missing meta link: fillable.
    qudfwe is missing meta link: fillable.
    wtnrwl is missing meta link: fillable.
    apjbrw is missing meta link: fillable.
    vzzafs is missing meta link: fillable.

  ### (.*) has no collision mesh. Create a collision mesh.: 2
    top_cabinet-zjtaus-0-base_link has no collision mesh. Create a collision mesh.
    public_trash_can-pdmzhv-0-base_link has no collision mesh. Create a collision mesh.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID glzckq requires joints but has no joints.

  ### Object (.*) has .* fillable meshes. Should have no more than one.: 1
    Object bowl-kthvrl-0-base_link has 2 fillable meshes. Should have no more than one.


## objects/legacy_batch-05
  ### (.*) is missing meta link: fillable.: 31
    clziqw is missing meta link: fillable.
    tahafx is missing meta link: fillable.
    rwuazb is missing meta link: fillable.
    qygclv is missing meta link: fillable.
    qriitd is missing meta link: fillable.
    xhdtjn is missing meta link: fillable.
    wnvxme is missing meta link: fillable.
    wdsnjy is missing meta link: fillable.
    vehhll is missing meta link: fillable.
    tllnvs is missing meta link: fillable.
    ptxdid is missing meta link: fillable.
    wuinhm is missing meta link: fillable.
    fexqbj is missing meta link: fillable.
    nkxhvf is missing meta link: fillable.
    vgzdul is missing meta link: fillable.
    lbbims is missing meta link: fillable.
    njwsoa is missing meta link: fillable.
    wqxrbf is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    nvqqkv is missing meta link: fillable.
    cqcoox is missing meta link: fillable.
    uriudf is missing meta link: fillable.
    hxorzh is missing meta link: fillable.
    pgwdlg is missing meta link: fillable.
    kfftgk is missing meta link: fillable.
    pqzyxx is missing meta link: fillable.
    eyidvz is missing meta link: fillable.
    pcyjbu is missing meta link: fillable.
    pkgbcp is missing meta link: fillable.
    xjftlf is missing meta link: fillable.
    kenajw is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 1
    muwcnf is missing meta link: togglebutton.


## objects/legacy_batch-06
  ### (.*) is missing meta link: fillable.: 25
    snsqrt is missing meta link: fillable.
    smcyys is missing meta link: fillable.
    rtfebu is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    juwaoh is missing meta link: fillable.
    jtqazu is missing meta link: fillable.
    hjjxmi is missing meta link: fillable.
    hivvdf is missing meta link: fillable.
    fhtvuq is missing meta link: fillable.
    ejooms is missing meta link: fillable.
    zexzrc is missing meta link: fillable.
    zlmnfg is missing meta link: fillable.
    dszchb is missing meta link: fillable.
    xyejdx is missing meta link: fillable.
    yfaufu is missing meta link: fillable.
    czyfhq is missing meta link: fillable.
    xiybkb is missing meta link: fillable.
    xsuyua is missing meta link: fillable.
    bnpjjy is missing meta link: fillable.
    wzpabm is missing meta link: fillable.
    abzvij is missing meta link: fillable.
    wivnic is missing meta link: fillable.
    vbquye is missing meta link: fillable.
    vuezel is missing meta link: fillable.
    bfbeeb is missing meta link: fillable.


## objects/legacy_batch-08
  ### (.*) is missing meta link: fillable.: 2
    wlilma is missing meta link: fillable.
    gbdzls is missing meta link: fillable.


## objects/legacy_batch-09
  ### (.*) is missing meta link: fillable.: 8
    slscza is missing meta link: fillable.
    sxlklf is missing meta link: fillable.
    piezyg is missing meta link: fillable.
    poqpux is missing meta link: fillable.
    nfoydb is missing meta link: fillable.
    otcmih is missing meta link: fillable.
    gvbiwl is missing meta link: fillable.
    acikzo is missing meta link: fillable.

  ### (.*) has no collision mesh. Create a collision mesh.: 1
    half_radish-zjeukz-0-base_link-Tsubpart has no collision mesh. Create a collision mesh.


## objects/legacy_batch-10
  ### (.*) is missing meta link: fillable.: 5
    pyttso is missing meta link: fillable.
    sfbdjn is missing meta link: fillable.
    ozykxk is missing meta link: fillable.
    oimydv is missing meta link: fillable.
    ambnzk is missing meta link: fillable.

  ### (.*) is missing meta link: heatsource.: 1
    ddlqye is missing meta link: heatsource.


## objects/legacy_batch-11
  ### (.*) is missing meta link: fillable.: 6
    bzsxgw is missing meta link: fillable.
    bbewjo is missing meta link: fillable.
    mcualj is missing meta link: fillable.
    fsinsu is missing meta link: fillable.
    iawoof is missing meta link: fillable.
    ppzttc is missing meta link: fillable.

  ### Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.: 2
    Cannot validate meta links and joints for model ID wbqjbt: Category cup not found in taxonomy.
    Cannot validate meta links and joints for model ID dhdhul: Category salad_bowl not found in taxonomy.

  ### Cannot validate clothness: category (.*) not found in taxonomy.: 2
    Cannot validate clothness: category salad_bowl not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.

  ### (.*) is in the deletion queue. Delete the object.: 2
    salad_bowl-dhdhul-0-base_link is in the deletion queue. Delete the object.
    cup-wbqjbt-0-base_link is in the deletion queue. Delete the object.


## objects/substances-02
  ### (.*) is in the deletion queue. Delete the object.: 2
    diced__beef_roast-vrzhwc-0 is in the deletion queue. Delete the object.
    clove-usxktf-0 is in the deletion queue. Delete the object.

  ### Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.: 1
    Cannot validate meta links and joints for model ID vrzhwc: Category diced__beef_roast not found in taxonomy.

  ### Cannot validate clothness: category (.*) not found in taxonomy.: 1
    Cannot validate clothness: category diced__beef_roast not found in taxonomy.


## objects/task_relavant-ab
  ### (.*) is missing meta link: fillable.: 2
    gqemcq is missing meta link: fillable.
    efkgcw is missing meta link: fillable.

  ### (.*) is in the deletion queue. Delete the object.: 1
    plate-efkgcw-0 is in the deletion queue. Delete the object.


## objects/task_relevant-xy
  ### (.*) is missing meta link: fillable.: 7
    yqtlhy is missing meta link: fillable.
    spppps is missing meta link: fillable.
    bgxzec is missing meta link: fillable.
    qtfzeq is missing meta link: fillable.
    vsqsuy is missing meta link: fillable.
    ksgizx is missing meta link: fillable.
    doiucd is missing meta link: fillable.


## scenes/Beechwood_0_garden
  ### (.*) is missing meta link: fillable.: 28
    rhohgs is missing meta link: fillable.
    qudfwe is missing meta link: fillable.
    bfbeeb is missing meta link: fillable.
    qohxjq is missing meta link: fillable.
    zexzrc is missing meta link: fillable.
    qesxhu is missing meta link: fillable.
    zcmnji is missing meta link: fillable.
    omeuop is missing meta link: fillable.
    qacthv is missing meta link: fillable.
    czyfhq is missing meta link: fillable.
    wuinhm is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    fwstpx is missing meta link: fillable.
    vgzdul is missing meta link: fillable.
    dszchb is missing meta link: fillable.
    njwsoa is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    ufhpbn is missing meta link: fillable.
    pluwfl is missing meta link: fillable.
    spojpj is missing meta link: fillable.
    znfvmj is missing meta link: fillable.
    fqhdne is missing meta link: fillable.
    immwzb is missing meta link: fillable.
    slgzfc is missing meta link: fillable.
    dmwxyl is missing meta link: fillable.
    nddvba is missing meta link: fillable.
    rvpunw is missing meta link: fillable.
    zlmnfg is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 13
    bfbeeb is missing meta link: togglebutton.
    czyfhq is missing meta link: togglebutton.
    jcbvfi is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    zexzrc is missing meta link: togglebutton.
    omeuop is missing meta link: togglebutton.
    ylvjhb is missing meta link: togglebutton.
    zlmnfg is missing meta link: togglebutton.
    pmntxh is missing meta link: togglebutton.
    vdxlda is missing meta link: togglebutton.
    wuinhm is missing meta link: togglebutton.
    znfvmj is missing meta link: togglebutton.
    ksecxq is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 6
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object immwzb instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object spojpj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fluidsink.: 3
    zexzrc is missing meta link: fluidsink.
    czyfhq is missing meta link: fluidsink.
    ksecxq is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 3
    czyfhq is missing meta link: fluidsource.
    zexzrc is missing meta link: fluidsource.
    ksecxq is missing meta link: fluidsource.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID qesxhu requires joints but has no joints.

  ### (.*) is missing meta link: heatsource.: 1
    pmntxh is missing meta link: heatsource.


## scenes/Beechwood_0_int
  ### (.*) is missing meta link: fillable.: 25
    omeuop is missing meta link: fillable.
    zexzrc is missing meta link: fillable.
    czyfhq is missing meta link: fillable.
    njwsoa is missing meta link: fillable.
    nddvba is missing meta link: fillable.
    bfbeeb is missing meta link: fillable.
    wuinhm is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    immwzb is missing meta link: fillable.
    qudfwe is missing meta link: fillable.
    vgzdul is missing meta link: fillable.
    fwstpx is missing meta link: fillable.
    qohxjq is missing meta link: fillable.
    qacthv is missing meta link: fillable.
    fqhdne is missing meta link: fillable.
    ufhpbn is missing meta link: fillable.
    znfvmj is missing meta link: fillable.
    dszchb is missing meta link: fillable.
    spojpj is missing meta link: fillable.
    dmwxyl is missing meta link: fillable.
    slgzfc is missing meta link: fillable.
    zlmnfg is missing meta link: fillable.
    pluwfl is missing meta link: fillable.
    rvpunw is missing meta link: fillable.
    owvfik is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 13
    czyfhq is missing meta link: togglebutton.
    omeuop is missing meta link: togglebutton.
    jcbvfi is missing meta link: togglebutton.
    pmntxh is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    wuinhm is missing meta link: togglebutton.
    zexzrc is missing meta link: togglebutton.
    zlmnfg is missing meta link: togglebutton.
    ksecxq is missing meta link: togglebutton.
    vdxlda is missing meta link: togglebutton.
    ylvjhb is missing meta link: togglebutton.
    znfvmj is missing meta link: togglebutton.
    bfbeeb is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 7
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object immwzb instances have different scales for base links. This may have broken things during the match links script.
    Articulated object spojpj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fluidsink.: 3
    czyfhq is missing meta link: fluidsink.
    zexzrc is missing meta link: fluidsink.
    ksecxq is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 3
    zexzrc is missing meta link: fluidsource.
    czyfhq is missing meta link: fluidsource.
    ksecxq is missing meta link: fluidsource.

  ### (.*) is missing meta link: heatsource.: 1
    pmntxh is missing meta link: heatsource.


## scenes/Beechwood_1_int
  ### (.*) is missing meta link: fillable.: 15
    fqhdne is missing meta link: fillable.
    nddvba is missing meta link: fillable.
    slgzfc is missing meta link: fillable.
    zexzrc is missing meta link: fillable.
    dmwxyl is missing meta link: fillable.
    kubcdk is missing meta link: fillable.
    pluwfl is missing meta link: fillable.
    rvpunw is missing meta link: fillable.
    bamfsz is missing meta link: fillable.
    jhymlr is missing meta link: fillable.
    jrhgeu is missing meta link: fillable.
    njwsoa is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    vgzdul is missing meta link: fillable.
    ksecxq is missing meta link: fillable.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 9
    Articulated object jhymlr instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object pluwfl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object kubcdk instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object kfmkbm instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: togglebutton.: 6
    xbfgjc is missing meta link: togglebutton.
    ksecxq is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    ylvjhb is missing meta link: togglebutton.
    zexzrc is missing meta link: togglebutton.
    vdxlda is missing meta link: togglebutton.

  ### (.*) is missing meta link: fluidsink.: 2
    zexzrc is missing meta link: fluidsink.
    ksecxq is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 2
    zexzrc is missing meta link: fluidsource.
    ksecxq is missing meta link: fluidsource.


## scenes/Benevolence_0_int
  ### (.*) is missing meta link: fillable.: 3
    fdjykf is missing meta link: fillable.
    zotrbg is missing meta link: fillable.
    ksecxq is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 2
    ksecxq is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.

  ### (.*) is missing meta link: fluidsink.: 1
    ksecxq is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 1
    ksecxq is missing meta link: fluidsource.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 1
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.


## scenes/Benevolence_1_int
  ### (.*) is missing meta link: fillable.: 16
    dmwxyl is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    qudfwe is missing meta link: fillable.
    ufhpbn is missing meta link: fillable.
    wuinhm is missing meta link: fillable.
    bfbeeb is missing meta link: fillable.
    czyfhq is missing meta link: fillable.
    lsyzkh is missing meta link: fillable.
    njwsoa is missing meta link: fillable.
    spojpj is missing meta link: fillable.
    vgzdul is missing meta link: fillable.
    vzzafs is missing meta link: fillable.
    xyejdx is missing meta link: fillable.
    eobsmt is missing meta link: fillable.
    fwstpx is missing meta link: fillable.
    dngvvi is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 7
    bfbeeb is missing meta link: togglebutton.
    czyfhq is missing meta link: togglebutton.
    ylvjhb is missing meta link: togglebutton.
    wuinhm is missing meta link: togglebutton.
    vdxlda is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    dngvvi is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 6
    Articulated object ufhpbn instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lsyzkh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fwstpx instances have different scales for base links. This may have broken things during the match links script.
    Articulated object eobsmt instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fluidsink.: 1
    czyfhq is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 1
    czyfhq is missing meta link: fluidsource.


## scenes/Benevolence_2_int
  ### (.*) is missing meta link: fillable.: 10
    oimydv is missing meta link: fillable.
    zlmnfg is missing meta link: fillable.
    fdjykf is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    slgzfc is missing meta link: fillable.
    zotrbg is missing meta link: fillable.
    jrhgeu is missing meta link: fillable.
    bamfsz is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    omeuop is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 6
    ksecxq is missing meta link: togglebutton.
    vdxlda is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    omeuop is missing meta link: togglebutton.
    zlmnfg is missing meta link: togglebutton.
    ylvjhb is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 2
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fluidsink.: 1
    ksecxq is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 1
    ksecxq is missing meta link: fluidsource.


## scenes/commercial_kitchen_fire_extinguisher
  ### (.*) is missing meta link: fillable.: 18
    ceaeqf is missing meta link: fillable.
    jtqazu is missing meta link: fillable.
    melxmd is missing meta link: fillable.
    ttmejh is missing meta link: fillable.
    xvchpe is missing meta link: fillable.
    yfycfx is missing meta link: fillable.
    bbjuqu is missing meta link: fillable.
    jqyhjy is missing meta link: fillable.
    notszn is missing meta link: fillable.
    pkdnbu is missing meta link: fillable.
    rgnjdt is missing meta link: fillable.
    xurqal is missing meta link: fillable.
    jhymlr is missing meta link: fillable.
    nmxgbm is missing meta link: fillable.
    vwylob is missing meta link: fillable.
    dvnwfa is missing meta link: fillable.
    cgtaer is missing meta link: fillable.
    mvsgdm is missing meta link: fillable.

  ### (.*) is missing meta link: heatsource.: 2
    wlzvfj is missing meta link: heatsource.
    ncdzkj is missing meta link: heatsource.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID fkfgvp requires joints but has no joints.

  ### (.*) has no collision mesh. Create a collision mesh.: 1
    C-lid-wtjthb-0 has no collision mesh. Create a collision mesh.


## scenes/commercial_kitchen_pans
  ### (.*) is missing meta link: fillable.: 7
    jpdtvg is missing meta link: fillable.
    amblrk is missing meta link: fillable.
    xuhsfd is missing meta link: fillable.
    xecfyh is missing meta link: fillable.
    ttmejh is missing meta link: fillable.
    jtqazu is missing meta link: fillable.
    pkdnbu is missing meta link: fillable.

  ### (.*) is missing meta link: heatsource.: 2
    szisej is missing meta link: heatsource.
    wchhzh is missing meta link: heatsource.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID ouzyvo requires joints but has no joints.


## scenes/gates_bedroom
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID xdxisn requires joints but has no joints.


## scenes/grocery_store_asian
  ### (.*) is missing meta link: fillable.: 20
    fgfrwb is missing meta link: fillable.
    ekvdjq is missing meta link: fillable.
    eeamky is missing meta link: fillable.
    dhattl is missing meta link: fillable.
    dtaove is missing meta link: fillable.
    bgupna is missing meta link: fillable.
    jqvfyl is missing meta link: fillable.
    akinbg is missing meta link: fillable.
    gihuth is missing meta link: fillable.
    afhwym is missing meta link: fillable.
    ydnnsy is missing meta link: fillable.
    yvydfm is missing meta link: fillable.
    xmjzos is missing meta link: fillable.
    xfmyyw is missing meta link: fillable.
    sylmbu is missing meta link: fillable.
    vhnlno is missing meta link: fillable.
    msngax is missing meta link: fillable.
    nocxjc is missing meta link: fillable.
    mqmadg is missing meta link: fillable.
    lleghp is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID eyegld requires joints but has no joints.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 1
    Articulated object xiajyb instances have different scales for base links. This may have broken things during the match links script.


## scenes/grocery_store_cafe
  ### (.*) is missing meta link: fillable.: 16
    dylnro is missing meta link: fillable.
    iwziew is missing meta link: fillable.
    kmgltg is missing meta link: fillable.
    tggwud is missing meta link: fillable.
    wranvj is missing meta link: fillable.
    iruewm is missing meta link: fillable.
    cjhskr is missing meta link: fillable.
    sjmdri is missing meta link: fillable.
    tfjzen is missing meta link: fillable.
    xngcbz is missing meta link: fillable.
    fwlabx is missing meta link: fillable.
    ldvnrg is missing meta link: fillable.
    prtqlw is missing meta link: fillable.
    rowulr is missing meta link: fillable.
    yursyw is missing meta link: fillable.
    fnoifb is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 3
    Model ID mefezc requires joints but has no joints.
    Model ID dyiufq requires joints but has no joints.
    Model ID fnoifb requires joints but has no joints.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 1
    Articulated object rxvopf instances have different scales for base links. This may have broken things during the match links script.


## scenes/grocery_store_convenience
  ### (.*) is missing meta link: fillable.: 15
    ihxrvr is missing meta link: fillable.
    dxwbae is missing meta link: fillable.
    vzwcql is missing meta link: fillable.
    qkkjwp is missing meta link: fillable.
    kmgltg is missing meta link: fillable.
    duckhq is missing meta link: fillable.
    cdhybw is missing meta link: fillable.
    pbjodr is missing meta link: fillable.
    vduyjh is missing meta link: fillable.
    xchqhl is missing meta link: fillable.
    ykfcaz is missing meta link: fillable.
    oddyhd is missing meta link: fillable.
    wlxyjn is missing meta link: fillable.
    xqmipa is missing meta link: fillable.
    bqpanz is missing meta link: fillable.

  ### (.*) is in the deletion queue. Delete the object.: 8
    C-kebab-cewhbv-4 is in the deletion queue. Delete the object.
    C-kebab-cewhbv-0 is in the deletion queue. Delete the object.
    C-kebab-cewhbv-5 is in the deletion queue. Delete the object.
    C-kebab-cewhbv-6 is in the deletion queue. Delete the object.
    C-kebab-cewhbv-7 is in the deletion queue. Delete the object.
    C-kebab-cewhbv-3 is in the deletion queue. Delete the object.
    C-kebab-cewhbv-2 is in the deletion queue. Delete the object.
    C-kebab-cewhbv-1 is in the deletion queue. Delete the object.

  ### Model ID (.*) requires joints but has no joints.: 3
    Model ID vzwcql requires joints but has no joints.
    Model ID uartvl requires joints but has no joints.
    Model ID xjyvwx requires joints but has no joints.

  ### (.*) is missing meta link: togglebutton.: 1
    fjpams is missing meta link: togglebutton.


## scenes/grocery_store_half_stocked
  ### (.*) is missing meta link: fillable.: 9
    wvkefp is missing meta link: fillable.
    nuzsvi is missing meta link: fillable.
    ztrupq is missing meta link: fillable.
    wiiyfk is missing meta link: fillable.
    znpari is missing meta link: fillable.
    rywinu is missing meta link: fillable.
    kpfgcm is missing meta link: fillable.
    zeoukc is missing meta link: fillable.
    xzusqq is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID rkqcfx requires joints but has no joints.
    Model ID crlhmi requires joints but has no joints.


## scenes/hall_arch_wood
  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 11
    ijnwlp link glass has different position in instance 11 compared to instance 0. Position difference: [-3.73193169e+00 -4.60644865e+00  2.36311462e-06].
    ijnwlp link glass has different position in instance 6 compared to instance 0. Position difference: [1.35134482e+00 1.70353699e+00 2.94575002e-05].
    ijnwlp link glass has different position in instance 7 compared to instance 0. Position difference: [1.35134482e+00 1.70353699e+00 2.94575002e-05].
    ijnwlp link glass has different position in instance 10 compared to instance 0. Position difference: [-3.73193288e+00 -4.60654020e+00  2.36276537e-06].
    ijnwlp link glass has different position in instance 4 compared to instance 0. Position difference: [ 1.56390977e+00  1.97361946e+00 -9.54524148e-06].
    ijnwlp link glass has different position in instance 5 compared to instance 0. Position difference: [ 1.56392503e+00  1.97361898e+00 -9.54524148e-06].
    ijnwlp link glass has different position in instance 3 compared to instance 0. Position difference: [ 1.56395555e+00  1.97361851e+00 -9.54535790e-06].
    ijnwlp link glass has different position in instance 1 compared to instance 0. Position difference: [-8.55597019e-01 -1.06878829e+00  2.97740335e-05].
    ijnwlp link glass has different position in instance 9 compared to instance 0. Position difference: [-3.73193216e+00 -4.60647917e+00  2.36299820e-06].
    ijnwlp link glass has different position in instance 2 compared to instance 0. Position difference: [ 1.56395173e+00  1.97361851e+00 -9.54535790e-06].
    ijnwlp link glass has different position in instance 8 compared to instance 0. Position difference: [1.35126853e+00 1.70353842e+00 2.94577330e-05].

  ### Model ID (.*) requires joints but has no joints.: 3
    Model ID ijnwlp requires joints but has no joints.
    Model ID kmvdjo requires joints but has no joints.
    Model ID yxclci requires joints but has no joints.


## scenes/hall_glass_ceiling
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID cggyho requires joints but has no joints.


## scenes/hall_train_station
  ### Model ID (.*) requires joints but has no joints.: 4
    Model ID vsifsu requires joints but has no joints.
    Model ID asrqdf requires joints but has no joints.
    Model ID vqbgzc requires joints but has no joints.
    Model ID cmuvzz requires joints but has no joints.


## scenes/hotel_gym_spa
  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 5
    ceddpg link link_1 has different rotation in instance 1 compared to instance 0. Rotation difference: 0.025410676825407177.
    lcrefl link leaf has different position in instance 1 compared to instance 0. Position difference: [-0.41162109  1.10595703  0.        ].
    dladgw link glass has different position in instance 1 compared to instance 0. Position difference: [-78.85980988  -0.88166004  68.59809113].
    ceddpg link link_2 has different rotation in instance 1 compared to instance 0. Rotation difference: 0.025410676825407177.
    dladgw link glass has different position in instance 2 compared to instance 0. Position difference: [-7.88129272e+01  1.75930560e-04  6.85988312e+01].

  ### (.*) is missing meta link: fillable.: 4
    buifoi is missing meta link: fillable.
    pbmwrv is missing meta link: fillable.
    apiaid is missing meta link: fillable.
    bxlkyo is missing meta link: fillable.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 3
    Articulated object nfrbch instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dladgw instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ceddpg instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) has different shear. Match scaling axes on each instance.: 2
    B-door-nfrbch-1-link_2-link_1-R-lower has different shear. Match scaling axes on each instance.
    B-door-nfrbch-1-link_1-base_link-R-lower has different shear. Match scaling axes on each instance.


## scenes/hotel_suite_large
  ### (.*) is missing meta link: fillable.: 6
    nbmidu is missing meta link: fillable.
    jbborl is missing meta link: fillable.
    rglkkf is missing meta link: fillable.
    cbidyu is missing meta link: fillable.
    aqtmhm is missing meta link: fillable.
    xogqqm is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID ucejof requires joints but has no joints.


## scenes/hotel_suite_small
  ### (.*) is missing meta link: fillable.: 1
    kxvbui is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID vuevwo requires joints but has no joints.


## scenes/house_double_floor_lower
  ### (.*) is missing meta link: fillable.: 12
    gwhcty is missing meta link: fillable.
    fqsjey is missing meta link: fillable.
    tjextj is missing meta link: fillable.
    ffitak is missing meta link: fillable.
    rhdbzv is missing meta link: fillable.
    todoxz is missing meta link: fillable.
    tynnnw is missing meta link: fillable.
    egwapq is missing meta link: fillable.
    byvbuc is missing meta link: fillable.
    qesxhu is missing meta link: fillable.
    vanltx is missing meta link: fillable.
    hecjnt is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 10
    Model ID pczjnr requires joints but has no joints.
    Model ID tjextj requires joints but has no joints.
    Model ID qesxhu requires joints but has no joints.
    Model ID tynnnw requires joints but has no joints.
    Model ID kvdqep requires joints but has no joints.
    Model ID lhzwtz requires joints but has no joints.
    Model ID ifgcmr requires joints but has no joints.
    Model ID todoxz requires joints but has no joints.
    Model ID xnjqix requires joints but has no joints.
    Model ID byvbuc requires joints but has no joints.

  ### Model ID (.*) contains 'todo'.: 1
    Model ID todoxz contains 'todo'.

  ### (.*) is missing meta link: togglebutton.: 1
    ffitak is missing meta link: togglebutton.

  ### (.*) is in the deletion queue. Delete the object.: 1
    F-bowl-gwhcty-0 is in the deletion queue. Delete the object.

  ### (.*) is missing meta link: heatsource.: 1
    sxfjac is missing meta link: heatsource.


## scenes/house_double_floor_upper
  ### (.*) is missing meta link: fillable.: 16
    bsvnni is missing meta link: fillable.
    fdqxyk is missing meta link: fillable.
    kdnyfh is missing meta link: fillable.
    kmuacd is missing meta link: fillable.
    qwskbo is missing meta link: fillable.
    szdjbr is missing meta link: fillable.
    tsjvyu is missing meta link: fillable.
    akdubn is missing meta link: fillable.
    sapyzl is missing meta link: fillable.
    eckdny is missing meta link: fillable.
    ncgmqr is missing meta link: fillable.
    ojjqku is missing meta link: fillable.
    opryhp is missing meta link: fillable.
    otwukr is missing meta link: fillable.
    ejxdjr is missing meta link: fillable.
    rufepf is missing meta link: fillable.

  ### (.*) is in the deletion queue. Delete the object.: 13
    C-folder-ejxdjr-2 is in the deletion queue. Delete the object.
    C-folder-ejxdjr-3 is in the deletion queue. Delete the object.
    C-folder-ejxdjr-9 is in the deletion queue. Delete the object.
    C-folder-ejxdjr-1 is in the deletion queue. Delete the object.
    C-folder-ejxdjr-5 is in the deletion queue. Delete the object.
    C-folder-ejxdjr-8 is in the deletion queue. Delete the object.
    C-sheet-vhjpzt-0 is in the deletion queue. Delete the object.
    C-sheet-ddayzw-0 is in the deletion queue. Delete the object.
    C-folder-ejxdjr-0 is in the deletion queue. Delete the object.
    C-folder-ejxdjr-7 is in the deletion queue. Delete the object.
    C-sheet-wbdatw-0 is in the deletion queue. Delete the object.
    C-folder-ejxdjr-6 is in the deletion queue. Delete the object.
    C-folder-ejxdjr-4 is in the deletion queue. Delete the object.

  ### Model ID (.*) requires joints but has no joints.: 7
    Model ID qwskbo requires joints but has no joints.
    Model ID fdqxyk requires joints but has no joints.
    Model ID akdubn requires joints but has no joints.
    Model ID lhzwtz requires joints but has no joints.
    Model ID ongghk requires joints but has no joints.
    Model ID ncgmqr requires joints but has no joints.
    Model ID etjpqr requires joints but has no joints.

  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 4
    pxhbim link leaf has different position in instance 1 compared to instance 0. Position difference: [-1.36022282e+00  1.68204327e+01 -7.04014781e-07].
    pxhbim link leaf has different rotation in instance 3 compared to instance 0. Rotation difference: 0.040591377225218765.
    pxhbim link leaf has different position in instance 3 compared to instance 0. Position difference: [-1.36034489e+00  1.68204327e+01 -7.04032971e-07].
    pxhbim link leaf has different rotation in instance 1 compared to instance 0. Rotation difference: 0.040591377225218765.

  ### Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.: 3
    Cannot validate meta links and joints for model ID ddayzw: Category sheet not found in taxonomy.
    Cannot validate meta links and joints for model ID wbdatw: Category sheet not found in taxonomy.
    Cannot validate meta links and joints for model ID vhjpzt: Category sheet not found in taxonomy.

  ### Cannot validate clothness: category (.*) not found in taxonomy.: 3
    Cannot validate clothness: category sheet not found in taxonomy.
    Cannot validate clothness: category sheet not found in taxonomy.
    Cannot validate clothness: category sheet not found in taxonomy.

  ### (.*) is missing meta link: togglebutton.: 1
    strbnw is missing meta link: togglebutton.


## scenes/house_single_floor
  ### (.*) is missing meta link: fillable.: 106
    bbduix is missing meta link: fillable.
    awvzkn is missing meta link: fillable.
    aviadj is missing meta link: fillable.
    amyper is missing meta link: fillable.
    amutun is missing meta link: fillable.
    adiwil is missing meta link: fillable.
    ktworp is missing meta link: fillable.
    lrokhl is missing meta link: fillable.
    krgqwl is missing meta link: fillable.
    sijioo is missing meta link: fillable.
    xkwxtj is missing meta link: fillable.
    kitxam is missing meta link: fillable.
    lkxmne is missing meta link: fillable.
    qnequy is missing meta link: fillable.
    xipfrv is missing meta link: fillable.
    sfkezf is missing meta link: fillable.
    xfjmld is missing meta link: fillable.
    pmhcse is missing meta link: fillable.
    itoeew is missing meta link: fillable.
    pfusrd is missing meta link: fillable.
    todoac is missing meta link: fillable.
    jzmrdd is missing meta link: fillable.
    sclnqc is missing meta link: fillable.
    wauyns is missing meta link: fillable.
    jtmpsj is missing meta link: fillable.
    pdcrfs is missing meta link: fillable.
    vztxva is missing meta link: fillable.
    djrdum is missing meta link: fillable.
    pjaljg is missing meta link: fillable.
    qjbbbb is missing meta link: fillable.
    vxqpnm is missing meta link: fillable.
    itlkfz is missing meta link: fillable.
    sakwru is missing meta link: fillable.
    saenda is missing meta link: fillable.
    vxobmt is missing meta link: fillable.
    dszchb is missing meta link: fillable.
    gwqitr is missing meta link: fillable.
    pyuwgc is missing meta link: fillable.
    vnvmkx is missing meta link: fillable.
    drxfmr is missing meta link: fillable.
    jaysra is missing meta link: fillable.
    rykngq is missing meta link: fillable.
    ggpnlr is missing meta link: fillable.
    nkzulg is missing meta link: fillable.
    tkgsho is missing meta link: fillable.
    vekaaf is missing meta link: fillable.
    dmizts is missing meta link: fillable.
    onezuj is missing meta link: fillable.
    rwotxo is missing meta link: fillable.
    vccsrl is missing meta link: fillable.
    rrmzvc is missing meta link: fillable.
    cmdagy is missing meta link: fillable.
    vbwsci is missing meta link: fillable.
    dgkhyn is missing meta link: fillable.
    gvygxu is missing meta link: fillable.
    uzdupi is missing meta link: fillable.
    czcujg is missing meta link: fillable.
    coqeme is missing meta link: fillable.
    mdtkkv is missing meta link: fillable.
    upxnpu is missing meta link: fillable.
    zwekzu is missing meta link: fillable.
    exvfii is missing meta link: fillable.
    nqiujp is missing meta link: fillable.
    upwldu is missing meta link: fillable.
    zlaser is missing meta link: fillable.
    euxylv is missing meta link: fillable.
    kfiujj is missing meta link: fillable.
    unasxd is missing meta link: fillable.
    cqoesa is missing meta link: fillable.
    molqhs is missing meta link: fillable.
    rkagxx is missing meta link: fillable.
    tyczoo is missing meta link: fillable.
    zfpyqe is missing meta link: fillable.
    bhyopq is missing meta link: fillable.
    fkghop is missing meta link: fillable.
    meetii is missing meta link: fillable.
    taixcy is missing meta link: fillable.
    ffitak is missing meta link: fillable.
    ovipql is missing meta link: fillable.
    stibna is missing meta link: fillable.
    fjozhc is missing meta link: fillable.
    nfuxzd is missing meta link: fillable.
    qhdikw is missing meta link: fillable.
    todopi is missing meta link: fillable.
    ynwamu is missing meta link: fillable.
    dywzhj is missing meta link: fillable.
    jqfftz is missing meta link: fillable.
    otswdv is missing meta link: fillable.
    rfegnv is missing meta link: fillable.
    dwspgo is missing meta link: fillable.
    hitnkv is missing meta link: fillable.
    sstojv is missing meta link: fillable.
    todoph is missing meta link: fillable.
    bnobdx is missing meta link: fillable.
    gjrero is missing meta link: fillable.
    npuuir is missing meta link: fillable.
    ssfvij is missing meta link: fillable.
    ybzffr is missing meta link: fillable.
    jdwvyt is missing meta link: fillable.
    mrrpay is missing meta link: fillable.
    todopf is missing meta link: fillable.
    bbpraa is missing meta link: fillable.
    hibwic is missing meta link: fillable.
    ociqav is missing meta link: fillable.
    qwoqqr is missing meta link: fillable.
    xsuyua is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 32
    Model ID nruots requires joints but has no joints.
    Model ID qvqecb requires joints but has no joints.
    Model ID ropopf requires joints but has no joints.
    Model ID xkxvpp requires joints but has no joints.
    Model ID pjaljg requires joints but has no joints.
    Model ID eyvzfu requires joints but has no joints.
    Model ID iggzsd requires joints but has no joints.
    Model ID usynui requires joints but has no joints.
    Model ID ociqav requires joints but has no joints.
    Model ID hwunjt requires joints but has no joints.
    Model ID glimdy requires joints but has no joints.
    Model ID wogynv requires joints but has no joints.
    Model ID hohdkt requires joints but has no joints.
    Model ID ovzepp requires joints but has no joints.
    Model ID tspbac requires joints but has no joints.
    Model ID yzxhbp requires joints but has no joints.
    Model ID dfdweq requires joints but has no joints.
    Model ID izydvb requires joints but has no joints.
    Model ID tojcni requires joints but has no joints.
    Model ID engzvq requires joints but has no joints.
    Model ID vxqpnm requires joints but has no joints.
    Model ID jrnfkt requires joints but has no joints.
    Model ID yjbico requires joints but has no joints.
    Model ID yikxbf requires joints but has no joints.
    Model ID nszybf requires joints but has no joints.
    Model ID vhbsgv requires joints but has no joints.
    Model ID dbqbzg requires joints but has no joints.
    Model ID bvqijp requires joints but has no joints.
    Model ID qfifse requires joints but has no joints.
    Model ID xucvcj requires joints but has no joints.
    Model ID ijpvgf requires joints but has no joints.
    Model ID jdwvyt requires joints but has no joints.

  ### Model ID (.*) contains 'todo'.: 5
    Model ID todoad contains 'todo'.
    Model ID todoph contains 'todo'.
    Model ID todopi contains 'todo'.
    Model ID todopf contains 'todo'.
    Model ID todoac contains 'todo'.

  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 5
    kwbnhy link glass has different position in instance 3 compared to instance 0. Position difference: [-4.76074219e-03 -8.88276367e+01 -9.37734375e+01].
    kwbnhy link glass has different position in instance 2 compared to instance 0. Position difference: [-0.00720215 -3.78320312 -3.94335938].
    kwbnhy link leaf has different position in instance 1 compared to instance 0. Position difference: [2.83467770e-03 5.23731003e+01 5.42108917e+01].
    kwbnhy link leaf has different position in instance 3 compared to instance 0. Position difference: [-6.48624301e-02 -8.25469589e+01 -8.67889862e+01].
    kwbnhy link leaf has different position in instance 2 compared to instance 0. Position difference: [-3.55041027e-03 -1.98281441e+01 -2.06777172e+01].

  ### (.*) is in the deletion queue. Delete the object.: 3
    C-toy-wqzqoy-0 is in the deletion queue. Delete the object.
    C-toy-ywtond-0 is in the deletion queue. Delete the object.
    C-vase-sakwru-0 is in the deletion queue. Delete the object.

  ### (.*) is missing meta link: particleapplier.: 2
    fztfsi is missing meta link: particleapplier.
    zkpgkl is missing meta link: particleapplier.

  ### (.*) is missing meta link: togglebutton.: 2
    crdaeu is missing meta link: togglebutton.
    xsuyua is missing meta link: togglebutton.

  ### Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.: 2
    Cannot validate meta links and joints for model ID wqzqoy: Category toy not found in taxonomy.
    Cannot validate meta links and joints for model ID ywtond: Category toy not found in taxonomy.

  ### Cannot validate clothness: category (.*) not found in taxonomy.: 2
    Cannot validate clothness: category toy not found in taxonomy.
    Cannot validate clothness: category toy not found in taxonomy.

  ### (.*) has no collision mesh. Create a collision mesh.: 1
    C-toolbox-ybzffr-0 has no collision mesh. Create a collision mesh.

  ### (.*) is missing meta link: heatsource.: 1
    crdaeu is missing meta link: heatsource.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 1
    Articulated object gjrero instances have different scales for base links. This may have broken things during the match links script.


## scenes/Ihlen_0_int
  ### (.*) is missing meta link: fillable.: 7
    owvfik is missing meta link: fillable.
    omeuop is missing meta link: fillable.
    xsuyua is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    jrhgeu is missing meta link: fillable.
    vgzdul is missing meta link: fillable.
    njwsoa is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 4
    omeuop is missing meta link: togglebutton.
    ksecxq is missing meta link: togglebutton.
    xsuyua is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 3
    Articulated object ktydvs instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fluidsink.: 1
    ksecxq is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 1
    ksecxq is missing meta link: fluidsource.


## scenes/Ihlen_1_int
  ### (.*) is missing meta link: fillable.: 17
    slgzfc is missing meta link: fillable.
    jrhgeu is missing meta link: fillable.
    bamfsz is missing meta link: fillable.
    eobsmt is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    qohxjq is missing meta link: fillable.
    ufhpbn is missing meta link: fillable.
    czyfhq is missing meta link: fillable.
    dngvvi is missing meta link: fillable.
    fwstpx is missing meta link: fillable.
    njwsoa is missing meta link: fillable.
    xyejdx is missing meta link: fillable.
    dmwxyl is missing meta link: fillable.
    fdjykf is missing meta link: fillable.
    jvdbxh is missing meta link: fillable.
    wuinhm is missing meta link: fillable.
    bmsclc is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 8
    czyfhq is missing meta link: togglebutton.
    dngvvi is missing meta link: togglebutton.
    jqsuky is missing meta link: togglebutton.
    wntwxx is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    wuinhm is missing meta link: togglebutton.
    xbfgjc is missing meta link: togglebutton.
    ksecxq is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 8
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fwstpx instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xbfgjc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object jrhgeu instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fluidsink.: 2
    ksecxq is missing meta link: fluidsink.
    czyfhq is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 2
    ksecxq is missing meta link: fluidsource.
    czyfhq is missing meta link: fluidsource.

  ### (.*) is missing meta link: heatsource.: 1
    wntwxx is missing meta link: heatsource.


## scenes/Merom_0_garden
  ### (.*) is missing meta link: fillable.: 15
    fwstpx is missing meta link: fillable.
    jgethp is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    vdedzt is missing meta link: fillable.
    xiybkb is missing meta link: fillable.
    exasdr is missing meta link: fillable.
    immwzb is missing meta link: fillable.
    omeuop is missing meta link: fillable.
    zlmnfg is missing meta link: fillable.
    gewjwh is missing meta link: fillable.
    dajebq is missing meta link: fillable.
    uqmgtw is missing meta link: fillable.
    wklill is missing meta link: fillable.
    zotrbg is missing meta link: fillable.
    nyfebf is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 6
    udotid is missing meta link: togglebutton.
    xiybkb is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    zlmnfg is missing meta link: togglebutton.
    omeuop is missing meta link: togglebutton.
    snbvop is missing meta link: togglebutton.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID surlxy requires joints but has no joints.
    Model ID gewjwh requires joints but has no joints.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 2
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: heatsource.: 1
    tusmtd is missing meta link: heatsource.

  ### (.*) is missing meta link: fluidsink.: 1
    xiybkb is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 1
    xiybkb is missing meta link: fluidsource.


## scenes/Merom_0_int
  ### (.*) is missing meta link: fillable.: 11
    omeuop is missing meta link: fillable.
    zlmnfg is missing meta link: fillable.
    zotrbg is missing meta link: fillable.
    immwzb is missing meta link: fillable.
    fwstpx is missing meta link: fillable.
    dajebq is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    vdedzt is missing meta link: fillable.
    wklill is missing meta link: fillable.
    xiybkb is missing meta link: fillable.
    nyfebf is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 6
    omeuop is missing meta link: togglebutton.
    snbvop is missing meta link: togglebutton.
    xiybkb is missing meta link: togglebutton.
    udotid is missing meta link: togglebutton.
    zlmnfg is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 2
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fluidsink.: 1
    xiybkb is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 1
    xiybkb is missing meta link: fluidsource.


## scenes/Merom_1_int
  ### (.*) is missing meta link: fillable.: 17
    fdjykf is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    wdsnjy is missing meta link: fillable.
    czyfhq is missing meta link: fillable.
    dngvvi is missing meta link: fillable.
    jhymlr is missing meta link: fillable.
    jrhgeu is missing meta link: fillable.
    vgzdul is missing meta link: fillable.
    xiybkb is missing meta link: fillable.
    dmwxyl is missing meta link: fillable.
    dajebq is missing meta link: fillable.
    clziqw is missing meta link: fillable.
    fqhdne is missing meta link: fillable.
    slgzfc is missing meta link: fillable.
    zotrbg is missing meta link: fillable.
    bamfsz is missing meta link: fillable.
    xyejdx is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 8
    czyfhq is missing meta link: togglebutton.
    xbfgjc is missing meta link: togglebutton.
    dngvvi is missing meta link: togglebutton.
    vdxlda is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    iqbpie is missing meta link: togglebutton.
    xiybkb is missing meta link: togglebutton.
    rgpphy is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 7
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xbfgjc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object slgzfc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fluidsink.: 2
    czyfhq is missing meta link: fluidsink.
    xiybkb is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 2
    czyfhq is missing meta link: fluidsource.
    xiybkb is missing meta link: fluidsource.

  ### (.*) is missing meta link: heatsource.: 1
    rgpphy is missing meta link: heatsource.


## scenes/office_bike
  ### (.*) is missing meta link: fillable.: 2
    mkxkwm is missing meta link: fillable.
    fvkjpt is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID mvutvn requires joints but has no joints.
    Model ID fqsuee requires joints but has no joints.


## scenes/office_cubicles_left
  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 9
    pwoerr link leaf has different position in instance 1 compared to instance 0. Position difference: [ 7.30590820e-02  3.35693359e-04 -2.78925934e+01].
    pwoerr link leaf has different position in instance 2 compared to instance 0. Position difference: [-4.57763672e-04  0.00000000e+00 -2.64570484e+01].
    pwoerr link leaf has different position in instance 8 compared to instance 0. Position difference: [ 3.59344482e-02 -8.54492188e-04 -2.72402515e+01].
    pwoerr link leaf has different position in instance 6 compared to instance 0. Position difference: [-4.57763672e-04  0.00000000e+00 -2.64570484e+01].
    pwoerr link leaf has different position in instance 3 compared to instance 0. Position difference: [-1.75628662e-02 -2.13623047e-04 -2.57382984e+01].
    pwoerr link leaf has different position in instance 5 compared to instance 0. Position difference: [ 3.61785889e-02 -1.22070312e-04 -2.72402515e+01].
    pwoerr link leaf has different position in instance 7 compared to instance 0. Position difference: [ 7.28149414e-02 -3.05175781e-05 -2.78925934e+01].
    ggcyib link leaf has different position in instance 4 compared to instance 0. Position difference: [4.57763672e-05 3.66210938e-04 2.64589958e+01].
    ggcyib link leaf has different position in instance 2 compared to instance 0. Position difference: [-1.98364258e-04  3.66210938e-04  2.64589958e+01].

  ### (.*) is missing meta link: fillable.: 5
    rvunhj is missing meta link: fillable.
    immwzb is missing meta link: fillable.
    hrdeys is missing meta link: fillable.
    zjtaus is missing meta link: fillable.
    slgzfc is missing meta link: fillable.

  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 5
    B-conference_table-qzmjrj-2-base_link has different pivot offset position (by [1.96696472e+02 1.88917221e+03 7.81806477e-06]). Match pivots on each instance.
    B-conference_table-qzmjrj-5-base_link has different pivot offset position (by [1.96696586e+02 1.88917234e+03 8.17862139e-06]). Match pivots on each instance.
    B-conference_table-qzmjrj-1-base_link has different pivot offset position (by [1.96635997e+02 1.88910376e+03 6.73639181e-06]). Match pivots on each instance.
    B-conference_table-qzmjrj-4-base_link has different pivot offset position (by [ 1.96696775e+02  1.88917255e+03 -2.08173634e-06]). Match pivots on each instance.
    B-conference_table-qzmjrj-3-base_link has different pivot offset position (by [1.96696677e+02 1.88917244e+03 7.81806477e-06]). Match pivots on each instance.

  ### (.*) is missing meta link: togglebutton.: 4
    kmltvq is missing meta link: togglebutton.
    aoqsbu is missing meta link: togglebutton.
    hpkvem is missing meta link: togglebutton.
    rifpjo is missing meta link: togglebutton.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID yazaic requires joints but has no joints.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 1
    Articulated object immwzb instances have different scales for base links. This may have broken things during the match links script.


## scenes/office_cubicles_right
  ### (.*) is missing meta link: fillable.: 4
    immwzb is missing meta link: fillable.
    fqitbw is missing meta link: fillable.
    rvunhj is missing meta link: fillable.
    jhymlr is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 4
    zsquya is missing meta link: togglebutton.
    kpxrpb is missing meta link: togglebutton.
    uusnbr is missing meta link: togglebutton.
    odlphz is missing meta link: togglebutton.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID yazaic requires joints but has no joints.


## scenes/office_large
  ### (.*) is in the deletion queue. Delete the object.: 85
    C-mouse-ptwame-47 is in the deletion queue. Delete the object.
    C-mouse-ptwame-42 is in the deletion queue. Delete the object.
    C-mouse-ptwame-1 is in the deletion queue. Delete the object.
    C-mouse-ptwame-48 is in the deletion queue. Delete the object.
    C-mouse-ptwame-34 is in the deletion queue. Delete the object.
    C-mouse-ptwame-3 is in the deletion queue. Delete the object.
    C-mouse-ptwame-49 is in the deletion queue. Delete the object.
    C-mouse-ptwame-36 is in the deletion queue. Delete the object.
    C-mouse-ptwame-6 is in the deletion queue. Delete the object.
    C-mouse-ptwame-50 is in the deletion queue. Delete the object.
    C-mouse-ptwame-46 is in the deletion queue. Delete the object.
    C-mouse-ptwame-2 is in the deletion queue. Delete the object.
    C-mouse-ptwame-51 is in the deletion queue. Delete the object.
    C-mouse-ptwame-32 is in the deletion queue. Delete the object.
    C-mouse-ptwame-12 is in the deletion queue. Delete the object.
    C-mouse-ptwame-52 is in the deletion queue. Delete the object.
    C-mouse-ptwame-35 is in the deletion queue. Delete the object.
    C-mouse-ptwame-4 is in the deletion queue. Delete the object.
    C-cup-vlikrk-7 is in the deletion queue. Delete the object.
    C-mouse-ptwame-37 is in the deletion queue. Delete the object.
    C-mouse-ptwame-7 is in the deletion queue. Delete the object.
    C-mouse-ptwame-44 is in the deletion queue. Delete the object.
    C-mouse-ptwame-14 is in the deletion queue. Delete the object.
    C-cup-vlikrk-6 is in the deletion queue. Delete the object.
    C-mouse-ptwame-41 is in the deletion queue. Delete the object.
    C-mouse-ptwame-13 is in the deletion queue. Delete the object.
    C-mouse-ptwame-43 is in the deletion queue. Delete the object.
    C-mouse-ptwame-5 is in the deletion queue. Delete the object.
    C-cup-vlikrk-5 is in the deletion queue. Delete the object.
    C-mouse-ptwame-23 is in the deletion queue. Delete the object.
    C-mouse-ptwame-8 is in the deletion queue. Delete the object.
    C-mouse-ptwame-45 is in the deletion queue. Delete the object.
    C-mouse-ptwame-15 is in the deletion queue. Delete the object.
    C-cup-vlikrk-4 is in the deletion queue. Delete the object.
    C-mouse-ptwame-33 is in the deletion queue. Delete the object.
    C-mouse-ptwame-20 is in the deletion queue. Delete the object.
    C-cup-vlikrk-3 is in the deletion queue. Delete the object.
    C-mouse-ptwame-38 is in the deletion queue. Delete the object.
    C-mouse-ptwame-55 is in the deletion queue. Delete the object.
    C-mouse-ptwame-26 is in the deletion queue. Delete the object.
    C-mouse-ptwame-54 is in the deletion queue. Delete the object.
    C-cup-vlikrk-2 is in the deletion queue. Delete the object.
    C-mouse-ptwame-18 is in the deletion queue. Delete the object.
    C-mouse-ptwame-21 is in the deletion queue. Delete the object.
    C-mouse-ptwame-56 is in the deletion queue. Delete the object.
    C-cup-vlikrk-0 is in the deletion queue. Delete the object.
    C-mouse-ptwame-24 is in the deletion queue. Delete the object.
    C-mouse-ptwame-27 is in the deletion queue. Delete the object.
    C-cup-vlikrk-1 is in the deletion queue. Delete the object.
    C-mouse-ptwame-19 is in the deletion queue. Delete the object.
    C-mouse-ptwame-22 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-5 is in the deletion queue. Delete the object.
    C-mouse-ptwame-25 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-6 is in the deletion queue. Delete the object.
    C-mouse-ptwame-9 is in the deletion queue. Delete the object.
    C-mouse-ptwame-28 is in the deletion queue. Delete the object.
    C-mouse-ptwame-10 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-7 is in the deletion queue. Delete the object.
    C-mouse-ptwame-40 is in the deletion queue. Delete the object.
    C-mouse-ptwame-0 is in the deletion queue. Delete the object.
    C-mouse-ptwame-39 is in the deletion queue. Delete the object.
    C-mouse-ptwame-11 is in the deletion queue. Delete the object.
    C-mouse-ptwame-29 is in the deletion queue. Delete the object.
    C-mouse-ptwame-53 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-16 is in the deletion queue. Delete the object.
    C-mouse-ptwame-30 is in the deletion queue. Delete the object.
    C-mouse-ptwame-16 is in the deletion queue. Delete the object.
    C-mouse-ptwame-31 is in the deletion queue. Delete the object.
    C-mouse-ptwame-17 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-15 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-14 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-13 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-12 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-11 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-10 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-9 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-8 is in the deletion queue. Delete the object.
    C-cup-wfavfd-0 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-3 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-4 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-2 is in the deletion queue. Delete the object.
    C-cup-wfavfd-1 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-1 is in the deletion queue. Delete the object.
    C-cup-wfavfd-2 is in the deletion queue. Delete the object.
    C-cup-lfxtqa-0 is in the deletion queue. Delete the object.

  ### Cannot validate clothness: category (.*) not found in taxonomy.: 28
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.
    Cannot validate clothness: category cup not found in taxonomy.

  ### (.*) is missing meta link: fillable.: 11
    glzckq is missing meta link: fillable.
    jhymlr is missing meta link: fillable.
    bamfsz is missing meta link: fillable.
    ojceew is missing meta link: fillable.
    rvunhj is missing meta link: fillable.
    yyntym is missing meta link: fillable.
    zuwvdo is missing meta link: fillable.
    juwaoh is missing meta link: fillable.
    immwzb is missing meta link: fillable.
    slgzfc is missing meta link: fillable.
    dajebq is missing meta link: fillable.

  ### (.*) has scale that is not 1. Reset scale.: 8
    B-bottom_cabinet-zuwvdo-1-base_link has scale that is not 1. Reset scale.
    B-bottom_cabinet-zuwvdo-0-link_1-base_link-R-lower has scale that is not 1. Reset scale.
    B-bottom_cabinet-zuwvdo-2-link_1-base_link-R-lower has scale that is not 1. Reset scale.
    B-bottom_cabinet-zuwvdo-3-link_1-base_link-R-lower has scale that is not 1. Reset scale.
    B-bottom_cabinet-zuwvdo-0-base_link has scale that is not 1. Reset scale.
    B-bottom_cabinet-zuwvdo-1-link_1-base_link-R-lower has scale that is not 1. Reset scale.
    B-bottom_cabinet-zuwvdo-3-base_link has scale that is not 1. Reset scale.
    B-bottom_cabinet-zuwvdo-2-base_link has scale that is not 1. Reset scale.

  ### Model ID (.*) requires joints but has no joints.: 4
    Model ID immwzb requires joints but has no joints.
    Model ID glzckq requires joints but has no joints.
    Model ID ymwnju requires joints but has no joints.
    Model ID zlmfwf requires joints but has no joints.

  ### Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.: 3
    Cannot validate meta links and joints for model ID lfxtqa: Category cup not found in taxonomy.
    Cannot validate meta links and joints for model ID vlikrk: Category cup not found in taxonomy.
    Cannot validate meta links and joints for model ID wfavfd: Category cup not found in taxonomy.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 3
    Articulated object rvunhj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object bamfsz instances have different scales for base links. This may have broken things during the match links script.
    Articulated object jhymlr instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 2
    dqnbsj link leaf has different position in instance 1 compared to instance 0. Position difference: [8.30596387e-01 3.08849849e-07 1.73234677e+00].
    dqnbsj link leaf has different rotation in instance 1 compared to instance 0. Rotation difference: 0.020088253841147197.

  ### (.*) has different vertex count than recorded in provider.*: 2
    immwzb-0 has different vertex count than recorded in provider: 40 != 392.
    immwzb-1 has different vertex count than recorded in provider: 40 != 392.

  ### (.*) has different face count than recorded in provider.*: 2
    immwzb-1 has different face count than recorded in provider: 64 != 436.
    immwzb-0 has different face count than recorded in provider: 64 != 436.


## scenes/office_vendor_machine
  ### (.*) is missing meta link: fillable.: 6
    zdsmhe is missing meta link: fillable.
    ygrams is missing meta link: fillable.
    bamfsz is missing meta link: fillable.
    xsfeyi is missing meta link: fillable.
    umumgu is missing meta link: fillable.
    enzfco is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID ygrams requires joints but has no joints.


## scenes/Pomaria_0_garden
  ### (.*) is missing meta link: fillable.: 10
    spojpj is missing meta link: fillable.
    xezdng is missing meta link: fillable.
    zexzrc is missing meta link: fillable.
    qgfbdj is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    supuws is missing meta link: fillable.
    jhymlr is missing meta link: fillable.
    njwsoa is missing meta link: fillable.
    vgzdul is missing meta link: fillable.
    fdjykf is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 4
    wseglt is missing meta link: togglebutton.
    rbqanz is missing meta link: togglebutton.
    udotid is missing meta link: togglebutton.
    zexzrc is missing meta link: togglebutton.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID supuws requires joints but has no joints.
    Model ID ysgkig requires joints but has no joints.

  ### Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.: 2
    Cannot validate meta links and joints for model ID gbutve: Category toy not found in taxonomy.
    Cannot validate meta links and joints for model ID ciutkv: Category toy not found in taxonomy.

  ### Cannot validate clothness: category (.*) not found in taxonomy.: 2
    Cannot validate clothness: category toy not found in taxonomy.
    Cannot validate clothness: category toy not found in taxonomy.

  ### (.*) is in the deletion queue. Delete the object.: 2
    C-toy-ciutkv-0 is in the deletion queue. Delete the object.
    C-toy-gbutve-0 is in the deletion queue. Delete the object.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 2
    Articulated object spojpj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: heatsource.: 1
    cgobyh is missing meta link: heatsource.

  ### (.*) is missing meta link: fluidsink.: 1
    zexzrc is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 1
    zexzrc is missing meta link: fluidsource.


## scenes/Pomaria_0_int
  ### (.*) is missing meta link: fillable.: 7
    owvfik is missing meta link: fillable.
    njwsoa is missing meta link: fillable.
    zexzrc is missing meta link: fillable.
    jhymlr is missing meta link: fillable.
    fdjykf is missing meta link: fillable.
    vgzdul is missing meta link: fillable.
    spojpj is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 4
    rbqanz is missing meta link: togglebutton.
    udotid is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    zexzrc is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 4
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object spojpj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fluidsink.: 1
    zexzrc is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 1
    zexzrc is missing meta link: fluidsource.


## scenes/Pomaria_1_int
  ### (.*) is missing meta link: fillable.: 22
    vdedzt is missing meta link: fillable.
    zlmnfg is missing meta link: fillable.
    jrhgeu is missing meta link: fillable.
    dmwxyl is missing meta link: fillable.
    bmsclc is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    pluwfl is missing meta link: fillable.
    bfbeeb is missing meta link: fillable.
    omeuop is missing meta link: fillable.
    bamfsz is missing meta link: fillable.
    dngvvi is missing meta link: fillable.
    fexqbj is missing meta link: fillable.
    fqhdne is missing meta link: fillable.
    spojpj is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    zexzrc is missing meta link: fillable.
    lsyzkh is missing meta link: fillable.
    vzzafs is missing meta link: fillable.
    ufhpbn is missing meta link: fillable.
    xyejdx is missing meta link: fillable.
    qohxjq is missing meta link: fillable.
    vgzdul is missing meta link: fillable.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 11
    Articulated object dmwxyl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object spojpj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object bmsclc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lsyzkh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xyejdx instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fexqbj instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: togglebutton.: 9
    pmntxh is missing meta link: togglebutton.
    bfbeeb is missing meta link: togglebutton.
    zexzrc is missing meta link: togglebutton.
    zlmnfg is missing meta link: togglebutton.
    ksecxq is missing meta link: togglebutton.
    dngvvi is missing meta link: togglebutton.
    fexqbj is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    omeuop is missing meta link: togglebutton.

  ### (.*) is missing meta link: fluidsink.: 2
    ksecxq is missing meta link: fluidsink.
    zexzrc is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 2
    ksecxq is missing meta link: fluidsource.
    zexzrc is missing meta link: fluidsource.

  ### (.*) is missing meta link: heatsource.: 1
    pmntxh is missing meta link: heatsource.


## scenes/Pomaria_2_int
  ### (.*) is missing meta link: fillable.: 7
    owvfik is missing meta link: fillable.
    vgzdul is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    fdjykf is missing meta link: fillable.
    rntwkg is missing meta link: fillable.
    dmwxyl is missing meta link: fillable.
    qohxjq is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 4
    wseglt is missing meta link: togglebutton.
    vdxlda is missing meta link: togglebutton.
    ksecxq is missing meta link: togglebutton.
    udotid is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 4
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fluidsink.: 1
    ksecxq is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 1
    ksecxq is missing meta link: fluidsource.


## scenes/public_restroom_blue
  ### (.*) is missing meta link: fillable.: 1
    wavlkt is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID wkzfdu requires joints but has no joints.


## scenes/public_restroom_brown
  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID qjcodu requires joints but has no joints.


## scenes/public_restroom_white
  ### (.*) is missing meta link: fillable.: 1
    wqjwbx is missing meta link: fillable.


## scenes/restaurant_asian
  ### (.*) is missing meta link: fillable.: 2
    kvlewx is missing meta link: fillable.
    seyhuo is missing meta link: fillable.

  ### (.*) has no collision mesh. Create a collision mesh.: 1
    C-bottle_of_sake-zrkfim-0 has no collision mesh. Create a collision mesh.


## scenes/restaurant_brunch
  ### (.*) is missing meta link: fillable.: 8
    ibrbnl is missing meta link: fillable.
    adgrkb is missing meta link: fillable.
    hrfwyd is missing meta link: fillable.
    zadqqv is missing meta link: fillable.
    inkqch is missing meta link: fillable.
    qrmiur is missing meta link: fillable.
    xrnzjv is missing meta link: fillable.
    cyckht is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 3
    Model ID zdeyzf requires joints but has no joints.
    Model ID waousd requires joints but has no joints.
    Model ID ywetvt requires joints but has no joints.


## scenes/restaurant_cafeteria
  ### Model ID (.*) requires joints but has no joints.: 8
    Model ID whujjc requires joints but has no joints.
    Model ID cciwhk requires joints but has no joints.
    Model ID lvuvbf requires joints but has no joints.
    Model ID oszzno requires joints but has no joints.
    Model ID djqiqc requires joints but has no joints.
    Model ID guhrnv requires joints but has no joints.
    Model ID cgchwh requires joints but has no joints.
    Model ID ayvcgc requires joints but has no joints.

  ### (.*) is missing meta link: fillable.: 3
    jtqazu is missing meta link: fillable.
    juwaoh is missing meta link: fillable.
    lsyzkh is missing meta link: fillable.

  ### (.*) is missing meta link: heatsource.: 1
    drcurn is missing meta link: heatsource.


## scenes/restaurant_diner
  ### (.*) is missing meta link: fillable.: 2
    juwaoh is missing meta link: fillable.
    olgoza is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID iueiur requires joints but has no joints.
    Model ID caymim requires joints but has no joints.


## scenes/restaurant_hotel
  ### (.*) is missing meta link: fillable.: 1
    ubsonj is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 1
    Model ID ozlemc requires joints but has no joints.


## scenes/restaurant_urban
  ### (.*) is missing meta link: fillable.: 5
    iofciz is missing meta link: fillable.
    hrrrbw is missing meta link: fillable.
    ekkwsy is missing meta link: fillable.
    hlgjme is missing meta link: fillable.
    tgcuwh is missing meta link: fillable.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID txxinx requires joints but has no joints.
    Model ID vapbnz requires joints but has no joints.

  ### (.*) is missing meta link: togglebutton.: 1
    ekkwsy is missing meta link: togglebutton.

  ### (.*) is missing meta link: fluidsink.: 1
    ekkwsy is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 1
    ekkwsy is missing meta link: fluidsource.


## scenes/Rs_garden
  ### (.*) is missing meta link: fillable.: 16
    abzvij is missing meta link: fillable.
    lsyzkh is missing meta link: fillable.
    njwsoa is missing meta link: fillable.
    slgzfc is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    xlmier is missing meta link: fillable.
    xyejdx is missing meta link: fillable.
    jhymlr is missing meta link: fillable.
    fqhdne is missing meta link: fillable.
    uobdoq is missing meta link: fillable.
    wuinhm is missing meta link: fillable.
    zexzrc is missing meta link: fillable.
    zotrbg is missing meta link: fillable.
    bamfsz is missing meta link: fillable.
    dajebq is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 11
    abzvij is missing meta link: togglebutton.
    xlmier is missing meta link: togglebutton.
    zexzrc is missing meta link: togglebutton.
    ksecxq is missing meta link: togglebutton.
    vdxlda is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    xbfgjc is missing meta link: togglebutton.
    nvulcs is missing meta link: togglebutton.
    udotid is missing meta link: togglebutton.
    wuinhm is missing meta link: togglebutton.
    bmpdyv is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 4
    Articulated object slgzfc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lsyzkh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object bamfsz instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fluidsink.: 2
    zexzrc is missing meta link: fluidsink.
    ksecxq is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 2
    zexzrc is missing meta link: fluidsource.
    ksecxq is missing meta link: fluidsource.

  ### (.*) is missing meta link: heatsource.: 1
    dtfail is missing meta link: heatsource.


## scenes/Rs_int
  ### (.*) is missing meta link: fillable.: 15
    abzvij is missing meta link: fillable.
    jhymlr is missing meta link: fillable.
    slgzfc is missing meta link: fillable.
    fqhdne is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    xyejdx is missing meta link: fillable.
    zotrbg is missing meta link: fillable.
    dajebq is missing meta link: fillable.
    lsyzkh is missing meta link: fillable.
    njwsoa is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    wuinhm is missing meta link: fillable.
    xlmier is missing meta link: fillable.
    zexzrc is missing meta link: fillable.
    bamfsz is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 11
    ksecxq is missing meta link: togglebutton.
    abzvij is missing meta link: togglebutton.
    udotid is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    zexzrc is missing meta link: togglebutton.
    vdxlda is missing meta link: togglebutton.
    xbfgjc is missing meta link: togglebutton.
    wuinhm is missing meta link: togglebutton.
    xlmier is missing meta link: togglebutton.
    nvulcs is missing meta link: togglebutton.
    bmpdyv is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 4
    Articulated object lsyzkh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ithrgo instances have different scales for base links. This may have broken things during the match links script.
    Articulated object bamfsz instances have different scales for base links. This may have broken things during the match links script.
    Articulated object slgzfc instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fluidsink.: 2
    zexzrc is missing meta link: fluidsink.
    ksecxq is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 2
    zexzrc is missing meta link: fluidsource.
    ksecxq is missing meta link: fluidsource.


## scenes/school_biology
  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 39
    B-window-rgmujm-8-base_link has different pivot offset position (by [-0.02543516  2.70997534  0.39261162]). Match pivots on each instance.
    B-window-rgmujm-1-glass_moving-moving-F-lower has different pivot offset position (by [-0.0265915   2.76362584  0.44239625]). Match pivots on each instance.
    B-window-rgmujm-2-glass-base_link-F-lower has different pivot offset position (by [-0.02585124  2.76353442  0.4427237 ]). Match pivots on each instance.
    B-window-rgmujm-6-base_link has different pivot offset position (by [-0.02474965  2.70879513  0.39263477]). Match pivots on each instance.
    B-window-rgmujm-3-glass-base_link-F-lower has different pivot offset position (by [-0.02585344  2.76035902  0.4431706 ]). Match pivots on each instance.
    B-window-rgmujm-4-base_link has different pivot offset position (by [-0.02580665  2.76733346  0.40117839]). Match pivots on each instance.
    B-window-rgmujm-2-base_link has different pivot offset position (by [-0.02525913  2.7667027   0.40137836]). Match pivots on each instance.
    B-window-rgmujm-4-glass-base_link-F-lower has different pivot offset position (by [-0.02635239  2.76392761  0.44271803]). Match pivots on each instance.
    B-window-rgmujm-6-glass_moving-moving-F-lower has different pivot offset position (by [-0.02538381  2.70624387  0.43522752]). Match pivots on each instance.
    B-window-rgmujm-7-base_link has different pivot offset position (by [-0.02469704  2.7391905   0.39652653]). Match pivots on each instance.
    B-window-rgmujm-2-glass_moving-moving-F-lower has different pivot offset position (by [-0.02592597  2.76433406  0.44257256]). Match pivots on each instance.
    B-window-rgmujm-8-glass_moving-moving-F-lower has different pivot offset position (by [-0.02607134  2.70712903  0.43519393]). Match pivots on each instance.
    B-window-rgmujm-1-glass-base_link-F-lower has different pivot offset position (by [-0.02651677  2.76285571  0.44253989]). Match pivots on each instance.
    B-door-xxipyh-1-base_link has different pivot offset position (by [-1.33284417e+00  8.61478489e-04 -9.45814202e+00]). Match pivots on each instance.
    B-window-rgmujm-5-base_link has different pivot offset position (by [-0.02520402  2.70481523  0.39257323]). Match pivots on each instance.
    B-window-rgmujm-7-glass-base_link-F-lower has different pivot offset position (by [-0.02531316  2.73567135  0.43787599]). Match pivots on each instance.
    B-window-rgmujm-8-moving-base_link-P-lower has different pivot offset position (by [-0.02545708  2.70987196  0.39273343]). Match pivots on each instance.
    B-door-xxipyh-2-base_link has different pivot offset position (by [-9.27627658e+00  9.19020755e-04 -6.57349781e+01]). Match pivots on each instance.
    B-window-rgmujm-3-base_link has different pivot offset position (by [-0.02538819  2.76296399  0.40029582]). Match pivots on each instance.
    B-window-rgmujm-8-glass-base_link-F-lower has different pivot offset position (by [-0.02588581  2.70630817  0.43335731]). Match pivots on each instance.
    B-window-rgmujm-3-glass_moving-moving-F-lower has different pivot offset position (by [-0.02602781  2.76008808  0.44304146]). Match pivots on each instance.
    B-window-rgmujm-6-moving-base_link-P-lower has different pivot offset position (by [-0.02478264  2.70892779  0.39275651]). Match pivots on each instance.
    B-window-rgmujm-6-glass-base_link-F-lower has different pivot offset position (by [-0.02532023  2.70542301  0.43339088]). Match pivots on each instance.
    B-window-rgmujm-1-base_link has different pivot offset position (by [-0.02589371  2.76664368  0.40121684]). Match pivots on each instance.
    B-door-xxipyh-2-reddoor2-base_link-R-lower has different pivot offset position (by [-1.00059661e-01  5.97516662e-05 -2.45824161e-04]). Match pivots on each instance.
    B-door-xxipyh-2-glass2-reddoor2-F-lower has different pivot offset position (by [-6.70835288e-01 -3.50380261e-05  1.48748897e-04]). Match pivots on each instance.
    B-window-rgmujm-7-glass_moving-moving-F-lower has different pivot offset position (by [-0.02540314  2.73655124  0.43774767]). Match pivots on each instance.
    B-door-xxipyh-2-glass1-reddoor1-F-lower has different pivot offset position (by [-6.70914003e-01  6.07568997e-04  2.07773061e-04]). Match pivots on each instance.
    B-window-rgmujm-5-moving-base_link-P-lower has different pivot offset position (by [-0.02512342  2.70522652  0.39239802]). Match pivots on each instance.
    B-window-rgmujm-4-moving-base_link-P-lower has different pivot offset position (by [-0.0258743   2.76760543  0.40136867]). Match pivots on each instance.
    B-window-rgmujm-5-glass-base_link-F-lower has different pivot offset position (by [-0.02571436  2.70135033  0.43273808]). Match pivots on each instance.
    B-window-rgmujm-3-moving-base_link-P-lower has different pivot offset position (by [-0.02544059  2.76329498  0.40028816]). Match pivots on each instance.
    B-door-xxipyh-1-glass-base_link-F-lower has different pivot offset position (by [-1.33416362e+00 -1.51912371e-03 -9.44642363e+00]). Match pivots on each instance.
    B-window-rgmujm-4-glass_moving-moving-F-lower has different pivot offset position (by [-0.02644237  2.7645502   0.44356445]). Match pivots on each instance.
    B-door-xxipyh-2-glass-base_link-F-lower has different pivot offset position (by [-9.28255284e+00 -4.85102702e-04 -6.57232583e+01]). Match pivots on each instance.
    B-window-rgmujm-2-moving-base_link-P-lower has different pivot offset position (by [-0.02531153  2.76697468  0.40135539]). Match pivots on each instance.
    B-window-rgmujm-1-moving-base_link-P-lower has different pivot offset position (by [-0.02607912  2.76703369  0.40120918]). Match pivots on each instance.
    B-window-rgmujm-5-glass_moving-moving-F-lower has different pivot offset position (by [-0.02567123  2.7022597   0.43260133]). Match pivots on each instance.
    B-window-rgmujm-7-moving-base_link-P-lower has different pivot offset position (by [-0.02482566  2.73913789  0.39644263]). Match pivots on each instance.

  ### (.*) is missing meta link: fillable.: 12
    fjytro is missing meta link: fillable.
    ldmnwh is missing meta link: fillable.
    mfyvai is missing meta link: fillable.
    vnmcfg is missing meta link: fillable.
    bpwjxr is missing meta link: fillable.
    mcukuh is missing meta link: fillable.
    tgodzn is missing meta link: fillable.
    yhthgr is missing meta link: fillable.
    gemgfz is missing meta link: fillable.
    gxajos is missing meta link: fillable.
    lxznit is missing meta link: fillable.
    qbdweu is missing meta link: fillable.

  ### (.*) is in the deletion queue. Delete the object.: 7
    B-C-test_tube-vnmcfg-5 is in the deletion queue. Delete the object.
    B-C-test_tube-vnmcfg-4 is in the deletion queue. Delete the object.
    B-C-test_tube-vnmcfg-1 is in the deletion queue. Delete the object.
    B-C-test_tube-vnmcfg-3 is in the deletion queue. Delete the object.
    B-C-test_tube-tgodzn-0 is in the deletion queue. Delete the object.
    B-C-test_tube-vnmcfg-2 is in the deletion queue. Delete the object.
    B-C-test_tube-vnmcfg-0 is in the deletion queue. Delete the object.

  ### (.*) is missing meta link: togglebutton.: 6
    bftrar is missing meta link: togglebutton.
    tiscsa is missing meta link: togglebutton.
    ldrjoi is missing meta link: togglebutton.
    qebtel is missing meta link: togglebutton.
    xijidt is missing meta link: togglebutton.
    wxkuww is missing meta link: togglebutton.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID khxpgc requires joints but has no joints.
    Model ID uztisk requires joints but has no joints.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 2
    Articulated object xxipyh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object rgmujm instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: heatsource.: 1
    bftrar is missing meta link: heatsource.


## scenes/school_chemistry
  ### (.*) is in the deletion queue. Delete the object.: 118
    C-test_tube-qwtyqj-9 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-20 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-34 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-36 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-1 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-30 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-21 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-18 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-16 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-7 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-11 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-15 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-32 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-37 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-54 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-17 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-22 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-16 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-38 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-41 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-23 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-17 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-12 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-37 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-39 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-57 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-24 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-19 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-26 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-39 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-1 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-59 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-25 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-13 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-2 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-26 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-3 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-0-Tglass is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-27 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-14 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-35 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-4 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-55 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-28 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-36 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-5 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-56 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-29 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-18 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-15 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-38 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-6 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-58 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-31 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-20 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-8 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-40 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-33 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-21 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-33 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-22 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-10 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-2 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-35 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-42 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-53 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-14 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-23 is in the deletion queue. Delete the object.
    C-test_tube-vnmcfg-3 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-3 is in the deletion queue. Delete the object.
    C-test_tube-vnmcfg-5 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-45 is in the deletion queue. Delete the object.
    C-test_tube-vnmcfg-2 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-46 is in the deletion queue. Delete the object.
    C-test_tube-ykvekt-0-Tglass is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-28 is in the deletion queue. Delete the object.
    C-test_tube-ykvekt-5 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-47 is in the deletion queue. Delete the object.
    C-test_tube-ykvekt-2 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-8 is in the deletion queue. Delete the object.
    C-test_tube-ykvekt-4 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-30 is in the deletion queue. Delete the object.
    C-test_tube-iejmzf-3 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-49 is in the deletion queue. Delete the object.
    C-test_tube-iejmzf-2 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-51 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-50 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-11 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-12 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-43 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-52 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-24 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-13 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-4 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-34 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-44 is in the deletion queue. Delete the object.
    C-test_tube-vnmcfg-0-Tglass is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-25 is in the deletion queue. Delete the object.
    C-test_tube-vnmcfg-1 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-5 is in the deletion queue. Delete the object.
    C-test_tube-vnmcfg-4 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-27 is in the deletion queue. Delete the object.
    C-test_tube-vnmcfg-6 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-6 is in the deletion queue. Delete the object.
    C-test_tube-ykvekt-1 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-7 is in the deletion queue. Delete the object.
    C-test_tube-ykvekt-3 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-29 is in the deletion queue. Delete the object.
    C-test_tube-ykvekt-6 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-48 is in the deletion queue. Delete the object.
    C-test_tube-iejmzf-0-Tglass is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-9 is in the deletion queue. Delete the object.
    C-test_tube-iejmzf-1 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-31 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-0-Tglass is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-10 is in the deletion queue. Delete the object.
    C-test_tube-qwtyqj-19 is in the deletion queue. Delete the object.
    C-test_tube-tgodzn-32 is in the deletion queue. Delete the object.

  ### (.*) is missing meta link: fillable.: 10
    inkwmw is missing meta link: fillable.
    gemgfz is missing meta link: fillable.
    mcukuh is missing meta link: fillable.
    rmmvcd is missing meta link: fillable.
    xgvyyv is missing meta link: fillable.
    yhthgr is missing meta link: fillable.
    lufqkq is missing meta link: fillable.
    tqyiso is missing meta link: fillable.
    gsksby is missing meta link: fillable.
    nitufd is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 10
    bftrar is missing meta link: togglebutton.
    qebtel is missing meta link: togglebutton.
    hylkmj is missing meta link: togglebutton.
    wxkuww is missing meta link: togglebutton.
    oedwla is missing meta link: togglebutton.
    ldrjoi is missing meta link: togglebutton.
    tiscsa is missing meta link: togglebutton.
    xjupjc is missing meta link: togglebutton.
    xijidt is missing meta link: togglebutton.
    tfgypq is missing meta link: togglebutton.

  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 4
    B-door-xxipyh-1-glass-base_link-F-lower has different pivot offset position (by [-1.35589452e+00 -5.42717029e-04 -9.61888410e+00]). Match pivots on each instance.
    B-door-xxipyh-2-glass-base_link-F-lower has different pivot offset position (by [-9.25269100e+00 -4.85175811e-04 -6.55664336e+01]). Match pivots on each instance.
    B-door-xxipyh-1-base_link has different pivot offset position (by [-1.35589452e+00 -1.15082497e-04 -9.61888410e+00]). Match pivots on each instance.
    B-door-xxipyh-2-base_link has different pivot offset position (by [-9.25269100e+00 -5.75412796e-05 -6.55664336e+01]). Match pivots on each instance.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 3
    Articulated object rgmujm instances have different scales for base links. This may have broken things during the match links script.
    Articulated object dnyzym instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xxipyh instances have different scales for base links. This may have broken things during the match links script.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID khxpgc requires joints but has no joints.
    Model ID uztisk requires joints but has no joints.

  ### (.*) is missing meta link: heatsource.: 1
    bftrar is missing meta link: heatsource.


## scenes/school_computer_lab_and_infirmary
  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 48
    rgmujm link glass has different position in instance 2 compared to instance 0. Position difference: [-113.1328125    -1.58123958   -6.09221745].
    rgmujm link glass has different position in instance 3 compared to instance 0. Position difference: [-113.1328125    -1.58221614   -6.09221745].
    uztisk link glass_right has different position in instance 10 compared to instance 0. Position difference: [ 5.56652695e-02 -1.06884754e+00 -2.44140625e-04].
    uztisk link glass_right has different position in instance 11 compared to instance 0. Position difference: [ 5.56652695e-02 -1.06884754e+00  2.04420275e-13].
    rgmujm link moving has different position in instance 6 compared to instance 0. Position difference: [ 2.34001465e+01 -1.22011833e+01  9.28249210e-06].
    rgmujm link glass has different position in instance 1 compared to instance 0. Position difference: [-113.1328125    -1.58612239   -6.09209538].
    uztisk link glass_mid has different position in instance 19 compared to instance 0. Position difference: [ 5.66418208e-02 -1.06140125e+00 -2.44140625e-04].
    uztisk link glass_mid has different position in instance 9 compared to instance 0. Position difference: [ 1.11330532e-01 -2.13024879e+00 -2.44140625e-04].
    rgmujm link moving has different position in instance 4 compared to instance 0. Position difference: [ 2.34003906e+01 -1.22011833e+01 -1.12787835e-04].
    rgmujm link moving has different position in instance 5 compared to instance 0. Position difference: [ 2.34023438e+01 -1.22021599e+01 -2.34857420e-04].
    uztisk link glass_mid has different position in instance 17 compared to instance 0. Position difference: [ 5.66418208e-02 -1.06140125e+00 -2.44140625e-04].
    uztisk link glass_mid has different position in instance 18 compared to instance 0. Position difference: [ 5.66418208e-02 -1.06127918e+00 -2.44140625e-04].
    rgmujm link moving has different position in instance 3 compared to instance 0. Position difference: [ 2.34003906e+01 -1.22021599e+01 -2.34857420e-04].
    uztisk link glass_mid has different position in instance 10 compared to instance 0. Position difference: [ 5.56652695e-02 -1.06884754e+00 -2.44140625e-04].
    uztisk link glass_mid has different position in instance 11 compared to instance 0. Position difference: [ 5.56652695e-02 -1.06884754e+00  2.04420275e-13].
    rgmujm link moving has different position in instance 1 compared to instance 0. Position difference: [ 2.33984375e+01 -1.22382927e+01 -4.78970731e-04].
    rgmujm link moving has different position in instance 2 compared to instance 0. Position difference: [ 2.34023438e+01 -1.22021599e+01 -3.56927747e-04].
    uztisk link glass_left has different position in instance 18 compared to instance 0. Position difference: [ 5.66418208e-02 -1.06127918e+00 -2.44140625e-04].
    uztisk link glass_left has different position in instance 9 compared to instance 0. Position difference: [ 1.11330532e-01 -2.13024879e+00 -2.44140625e-04].
    uztisk link glass_left has different position in instance 11 compared to instance 0. Position difference: [ 5.56652695e-02 -1.06884754e+00  2.04420275e-13].
    uztisk link glass_left has different position in instance 17 compared to instance 0. Position difference: [ 5.66418208e-02 -1.06140125e+00 -2.44140625e-04].
    uztisk link glass_left has different position in instance 10 compared to instance 0. Position difference: [ 5.56652695e-02 -1.06884754e+00 -2.44140625e-04].
    gemgfz link topdoor has different position in instance 7 compared to instance 0. Position difference: [ 4.22000885e-04 -4.75991328e-07 -8.85000671e+02].
    gemgfz link topdoor has different position in instance 2 compared to instance 0. Position difference: [ 4.22000885e-04 -4.75991328e-07 -8.85000854e+02].
    gemgfz link topdoor has different position in instance 23 compared to instance 0. Position difference: [ 4.22000885e-04 -4.75991328e-07 -8.85000854e+02].
    uztisk link glass_right has different position in instance 19 compared to instance 0. Position difference: [ 5.66418208e-02 -1.06140125e+00 -2.44140625e-04].
    gemgfz link topdoor has different position in instance 13 compared to instance 0. Position difference: [ 4.22000885e-04 -4.75991328e-07 -8.85000854e+02].
    gemgfz link topdoor has different position in instance 14 compared to instance 0. Position difference: [ 4.22000885e-04 -4.75991328e-07 -8.85000854e+02].
    gemgfz link bottomdoor has different position in instance 7 compared to instance 0. Position difference: [-4.22000885e-04 -1.21593817e-04  8.84999573e+02].
    gemgfz link topdoor has different position in instance 1 compared to instance 0. Position difference: [ 4.22000885e-04 -4.75991328e-07 -8.85000854e+02].
    rgmujm link glass_moving has different position in instance 2 compared to instance 0. Position difference: [-89.73236084 -13.78330612  -6.09232378].
    gemgfz link bottomdoor has different position in instance 2 compared to instance 0. Position difference: [-4.22000885e-04 -1.21593817e-04  8.84999573e+02].
    gemgfz link bottomdoor has different position in instance 23 compared to instance 0. Position difference: [-4.22000885e-04 -1.21593817e-04  8.84999573e+02].
    rgmujm link glass has different position in instance 4 compared to instance 0. Position difference: [-113.13378906   -1.58221614   -6.09209538].
    rgmujm link glass_moving has different position in instance 6 compared to instance 0. Position difference: [-89.73309326 -13.78428268  -6.09195757].
    uztisk link glass_right has different position in instance 18 compared to instance 0. Position difference: [ 5.66418208e-02 -1.06127918e+00 -2.44140625e-04].
    gemgfz link bottomdoor has different position in instance 14 compared to instance 0. Position difference: [-4.22000885e-04 -1.21593817e-04  8.84999573e+02].
    gemgfz link bottomdoor has different position in instance 13 compared to instance 0. Position difference: [-4.22000885e-04 -1.21593817e-04  8.84999573e+02].
    rgmujm link glass_moving has different position in instance 1 compared to instance 0. Position difference: [-89.73626709 -13.82432175  -6.09244585].
    rgmujm link glass_moving has different position in instance 5 compared to instance 0. Position difference: [-89.73236084 -13.78428268  -6.09232378].
    uztisk link glass_right has different position in instance 17 compared to instance 0. Position difference: [ 5.66418208e-02 -1.06140125e+00 -2.44140625e-04].
    gemgfz link bottomdoor has different position in instance 1 compared to instance 0. Position difference: [-4.22000885e-04 -1.21593817e-04  8.84999573e+02].
    rgmujm link glass has different position in instance 6 compared to instance 0. Position difference: [-113.13354492   -1.58221614   -6.0919733 ].
    rgmujm link glass_moving has different position in instance 4 compared to instance 0. Position difference: [-89.7333374  -13.78428268  -6.09207964].
    uztisk link glass_right has different position in instance 9 compared to instance 0. Position difference: [ 1.11330532e-01 -2.13024879e+00 -2.44140625e-04].
    rgmujm link glass has different position in instance 5 compared to instance 0. Position difference: [-113.1328125    -1.58221614   -6.09221745].
    rgmujm link glass_moving has different position in instance 3 compared to instance 0. Position difference: [-89.73431396 -13.78428268  -6.09232378].
    uztisk link glass_left has different position in instance 19 compared to instance 0. Position difference: [ 5.66418208e-02 -1.06140125e+00 -2.44140625e-04].

  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 10
    B-door-xxipyh-1-glass2-reddoor2-F-lower has different pivot offset position (by [1.01204127e-01 2.11528730e-06 2.24290930e-04]). Match pivots on each instance.
    B-door-xxipyh-3-glass-base_link-F-lower has different pivot offset position (by [0.2443324  0.         1.72495949]). Match pivots on each instance.
    B-door-xxipyh-3-base_link has different pivot offset position (by [0.2443324  0.         1.72495949]). Match pivots on each instance.
    B-door-xxipyh-1-glass1-reddoor1-F-lower has different pivot offset position (by [1.01230138e-01 4.75135920e-04 1.80930786e-05]). Match pivots on each instance.
    B-door-xxipyh-1-reddoor1-base_link-R-lower has different pivot offset position (by [1.01230114e-01 4.75135920e-04 3.84926831e-05]). Match pivots on each instance.
    B-door-xxipyh-1-glass-base_link-F-lower has different pivot offset position (by [9.39295483e+00 3.06340396e-04 6.65557253e+01]). Match pivots on each instance.
    B-door-xxipyh-1-base_link has different pivot offset position (by [9.39295483e+00 3.06340396e-04 6.65557253e+01]). Match pivots on each instance.
    B-door-xxipyh-1-reddoor2-base_link-R-lower has different pivot offset position (by [1.01196474e-01 2.11528730e-06 2.48912395e-04]). Match pivots on each instance.
    B-door-xxipyh-2-glass-base_link-F-lower has different pivot offset position (by [ 7.76011914e+00 -1.21293910e-04  5.49742212e+01]). Match pivots on each instance.
    B-door-xxipyh-2-base_link has different pivot offset position (by [ 7.76011914e+00 -1.21293910e-04  5.49742212e+01]). Match pivots on each instance.

  ### (.*) is missing meta link: fillable.: 9
    ccjinv is missing meta link: fillable.
    yhthgr is missing meta link: fillable.
    inkwmw is missing meta link: fillable.
    gemgfz is missing meta link: fillable.
    mcukuh is missing meta link: fillable.
    fkosow is missing meta link: fillable.
    eniafz is missing meta link: fillable.
    lufqkq is missing meta link: fillable.
    ckjxoo is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 6
    tiscsa is missing meta link: togglebutton.
    qebtel is missing meta link: togglebutton.
    tfgypq is missing meta link: togglebutton.
    xjupjc is missing meta link: togglebutton.
    xijidt is missing meta link: togglebutton.
    wxkuww is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 4
    Articulated object xxipyh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object rgmujm instances have different scales for base links. This may have broken things during the match links script.
    Articulated object nwlbit instances have different scales for base links. This may have broken things during the match links script.
    Articulated object uztisk instances have different scales for base links. This may have broken things during the match links script.

  ### Model ID (.*) requires joints but has no joints.: 2
    Model ID khxpgc requires joints but has no joints.
    Model ID uztisk requires joints but has no joints.


## scenes/school_geography
  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 34
    gemgfz link bottomdoor has different position in instance 8 compared to instance 0. Position difference: [ 3.74841690e-03 -7.87827361e-04  8.84996948e+02].
    uztisk link glass_left has different position in instance 11 compared to instance 0. Position difference: [ 1.11330524e-01 -2.12768531e+00 -2.44140625e-04].
    uztisk link glass_right has different position in instance 9 compared to instance 0. Position difference: [ 1.11330524e-01 -2.12768531e+00 -2.44140625e-04].
    gemgfz link bottomdoor has different position in instance 41 compared to instance 0. Position difference: [ 3.74841690e-03 -6.65756932e-04  8.84996948e+02].
    gemgfz link bottomdoor has different position in instance 44 compared to instance 0. Position difference: [ 3.74841690e-03 -6.65756932e-04  8.84996948e+02].
    uztisk link glass_left has different position in instance 10 compared to instance 0. Position difference: [ 1.11330524e-01 -2.12768531e+00  4.66023975e-13].
    uztisk link glass_right has different position in instance 27 compared to instance 0. Position difference: [ 5.66418208e-02 -1.05883777e+00  2.32390957e-13].
    gemgfz link bottomdoor has different position in instance 33 compared to instance 0. Position difference: [ 3.74841690e-03 -6.65756932e-04  8.84996948e+02].
    gemgfz link bottomdoor has different position in instance 38 compared to instance 0. Position difference: [ 3.74841690e-03 -6.65756932e-04  8.84996948e+02].
    uztisk link glass_right has different position in instance 26 compared to instance 0. Position difference: [ 5.46886958e-02 -1.05883777e+00  2.31641556e-13].
    gemgfz link bottomdoor has different position in instance 16 compared to instance 0. Position difference: [ 3.74841690e-03 -6.65756932e-04  8.84996948e+02].
    gemgfz link topdoor has different position in instance 8 compared to instance 0. Position difference: [ 4.06408310e-03  4.21606528e-04 -8.84998169e+02].
    uztisk link glass_right has different position in instance 25 compared to instance 0. Position difference: [ 5.46886958e-02 -1.05883777e+00  2.31641556e-13].
    gemgfz link topdoor has different position in instance 41 compared to instance 0. Position difference: [ 4.06408310e-03  5.43676782e-04 -8.84998169e+02].
    gemgfz link topdoor has different position in instance 44 compared to instance 0. Position difference: [ 4.06408310e-03  5.43676782e-04 -8.84998169e+02].
    uztisk link glass_right has different position in instance 11 compared to instance 0. Position difference: [ 1.11330524e-01 -2.12768531e+00 -2.44140625e-04].
    gemgfz link topdoor has different position in instance 38 compared to instance 0. Position difference: [ 4.06408310e-03  5.43676782e-04 -8.84998169e+02].
    gemgfz link topdoor has different position in instance 33 compared to instance 0. Position difference: [ 4.06408310e-03  5.43676666e-04 -8.84997986e+02].
    uztisk link glass_right has different position in instance 10 compared to instance 0. Position difference: [ 1.11330524e-01 -2.12768531e+00  4.66023975e-13].
    gemgfz link topdoor has different position in instance 16 compared to instance 0. Position difference: [ 4.06408310e-03  5.43676782e-04 -8.84998169e+02].
    uztisk link glass_mid has different position in instance 9 compared to instance 0. Position difference: [ 1.11330524e-01 -2.12768531e+00 -2.44140625e-04].
    ehwmol link door1 has different position in instance 1 compared to instance 0. Position difference: [-1.16295500e+01 -8.74582678e-02  8.87818984e-04].
    uztisk link glass_mid has different position in instance 27 compared to instance 0. Position difference: [ 5.66418208e-02 -1.05883777e+00  2.32390957e-13].
    uztisk link glass_mid has different position in instance 26 compared to instance 0. Position difference: [ 5.46886958e-02 -1.05883777e+00  2.31641556e-13].
    uztisk link glass_mid has different position in instance 25 compared to instance 0. Position difference: [ 5.46886958e-02 -1.05883777e+00  2.31641556e-13].
    uztisk link glass_mid has different position in instance 11 compared to instance 0. Position difference: [ 1.11330524e-01 -2.12768531e+00 -2.44140625e-04].
    xxipyh link glass has different position in instance 3 compared to instance 0. Position difference: [-1.3007822   4.86425781 -2.44042969].
    uztisk link glass_mid has different position in instance 10 compared to instance 0. Position difference: [ 1.11330524e-01 -2.12768531e+00  4.66023975e-13].
    xxipyh link glass has different position in instance 2 compared to instance 0. Position difference: [-0.79687542  0.80957019 -4.54638672].
    uztisk link glass_left has different position in instance 9 compared to instance 0. Position difference: [ 1.11330524e-01 -2.12768531e+00 -2.44140625e-04].
    xxipyh link glass has different position in instance 1 compared to instance 0. Position difference: [-0.81054729  0.81738269 -4.54638672].
    uztisk link glass_left has different position in instance 27 compared to instance 0. Position difference: [ 5.66418208e-02 -1.05883777e+00  2.32390957e-13].
    uztisk link glass_left has different position in instance 26 compared to instance 0. Position difference: [ 5.46886958e-02 -1.05883777e+00  2.31641556e-13].
    uztisk link glass_left has different position in instance 25 compared to instance 0. Position difference: [ 5.46886958e-02 -1.05883777e+00  2.31641556e-13].

  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 8
    B-window-rgmujm-9-glass-base_link-F-lower has different pivot offset position (by [ 0.00108913 -0.14144338 -0.02043393]). Match pivots on each instance.
    B-window-rgmujm-9-glass_moving-moving-F-lower has different pivot offset position (by [ 0.00108913 -0.14157452 -0.02046443]). Match pivots on each instance.
    B-window-rgmujm-10-glass-base_link-F-lower has different pivot offset position (by [ 0.00093077 -0.1411483  -0.02043393]). Match pivots on each instance.
    B-window-rgmujm-10-glass_moving-moving-F-lower has different pivot offset position (by [ 0.00090438 -0.14132198 -0.02048347]). Match pivots on each instance.
    B-window-rgmujm-7-glass-base_link-F-lower has different pivot offset position (by [ 0.00081406 -0.14088273 -0.02042635]). Match pivots on each instance.
    B-window-rgmujm-7-glass_moving-moving-F-lower has different pivot offset position (by [ 0.0008293  -0.14101387 -0.02045684]). Match pivots on each instance.
    B-window-rgmujm-8-glass-base_link-F-lower has different pivot offset position (by [ 0.00057653 -0.14033032 -0.02041492]). Match pivots on each instance.
    B-window-rgmujm-8-glass_moving-moving-F-lower has different pivot offset position (by [ 0.00056538 -0.14055478 -0.02047588]). Match pivots on each instance.

  ### (.*) is missing meta link: fillable.: 5
    inkwmw is missing meta link: fillable.
    mcukuh is missing meta link: fillable.
    lufqkq is missing meta link: fillable.
    yhthgr is missing meta link: fillable.
    gemgfz is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 5
    bftrar is missing meta link: togglebutton.
    xjupjc is missing meta link: togglebutton.
    ldrjoi is missing meta link: togglebutton.
    xijidt is missing meta link: togglebutton.
    tiscsa is missing meta link: togglebutton.

  ### Model ID (.*) requires joints but has no joints.: 3
    Model ID ebtgyg requires joints but has no joints.
    Model ID uztisk requires joints but has no joints.
    Model ID khxpgc requires joints but has no joints.

  ### (.*) is missing in instance .*, so relative transform check cannot be completed.: 3
    khxpgc link glass_front is missing in instance 1, so relative transform check cannot be completed.
    khxpgc link glass_left is missing in instance 1, so relative transform check cannot be completed.
    khxpgc link glass_right is missing in instance 1, so relative transform check cannot be completed.

  ### (.*) is missing meta link: heatsource.: 1
    bftrar is missing meta link: heatsource.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 1
    Articulated object rgmujm instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) has different vertex count than recorded in provider.*: 1
    khxpgc-1 has different vertex count than recorded in provider: 18925 != 19000.

  ### (.*) has different face count than recorded in provider.*: 1
    khxpgc-1 has different face count than recorded in provider: 34922 != 35060.

  ### Inconsistent link sets within model (.*): 1
    Inconsistent link sets within model ID khxpgc.


## scenes/school_gym
  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 12
    B-door-xxipyh-2-base_link has different pivot offset position (by [-1.63992503e+00 -4.85175524e-04 -1.16323006e+01]). Match pivots on each instance.
    B-door-xxipyh-3-glass1-reddoor1-F-lower has different pivot offset position (by [-6.62740722e-01  4.49444772e-04  6.96105671e-04]). Match pivots on each instance.
    B-door-xxipyh-3-glass2-reddoor2-F-lower has different pivot offset position (by [-0.66275409 -0.00066891 -0.00277827]). Match pivots on each instance.
    B-door-xxipyh-3-base_link has different pivot offset position (by [-9.16399776e+00  1.52549081e-03 -6.49431464e+01]). Match pivots on each instance.
    B-door-xxipyh-1-glass-base_link-F-lower has different pivot offset position (by [-1.64834897e+00 -2.25949299e-03 -1.16830804e+01]). Match pivots on each instance.
    B-door-xxipyh-1-glass1-reddoor1-F-lower has different pivot offset position (by [-0.11905614 -0.00071022 -0.00125448]). Match pivots on each instance.
    B-door-xxipyh-1-glass2-reddoor2-F-lower has different pivot offset position (by [-0.11910924 -0.00038168 -0.00102757]). Match pivots on each instance.
    B-door-xxipyh-2-glass-base_link-F-lower has different pivot offset position (by [-1.64045963e+00 -2.43822695e-03 -1.16244866e+01]). Match pivots on each instance.
    B-door-xxipyh-2-glass1-reddoor1-F-lower has different pivot offset position (by [-0.11848581 -0.00077682  0.00220266]). Match pivots on each instance.
    B-door-xxipyh-1-base_link has different pivot offset position (by [-1.64780945e+00 -3.06340396e-04 -1.16869881e+01]). Match pivots on each instance.
    B-door-xxipyh-2-glass2-reddoor2-F-lower has different pivot offset position (by [-0.11853894 -0.00027155 -0.00057246]). Match pivots on each instance.
    B-door-xxipyh-3-glass-base_link-F-lower has different pivot offset position (by [-9.16971698e+00 -1.40420594e-03 -6.49353332e+01]). Match pivots on each instance.

  ### Model ID (.*) requires joints but has no joints.: 5
    Model ID ebtgyg requires joints but has no joints.
    Model ID ocpgvi requires joints but has no joints.
    Model ID dompiq requires joints but has no joints.
    Model ID khxpgc requires joints but has no joints.
    Model ID uztisk requires joints but has no joints.

  ### (.*) is missing meta link: togglebutton.: 3
    xjupjc is missing meta link: togglebutton.
    qebtel is missing meta link: togglebutton.
    bftrar is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 3
    Articulated object rgmujm instances have different scales for base links. This may have broken things during the match links script.
    Articulated object nwlbit instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xxipyh instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: fillable.: 2
    yhthgr is missing meta link: fillable.
    gemgfz is missing meta link: fillable.

  ### (.*) is missing meta link: heatsource.: 1
    bftrar is missing meta link: heatsource.

  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 1
    ehwmol link door1 has different position in instance 1 compared to instance 0. Position difference: [ 1.25909977e+01 -1.06488816e-01 -1.53130677e-04].


## scenes/Wainscott_0_garden
  ### (.*) is missing meta link: fillable.: 26
    dngvvi is missing meta link: fillable.
    eobsmt is missing meta link: fillable.
    zexzrc is missing meta link: fillable.
    xxsgpq is missing meta link: fillable.
    czyfhq is missing meta link: fillable.
    jrhgeu is missing meta link: fillable.
    rixzrk is missing meta link: fillable.
    qohxjq is missing meta link: fillable.
    bmsclc is missing meta link: fillable.
    dszchb is missing meta link: fillable.
    jhymlr is missing meta link: fillable.
    pyttso is missing meta link: fillable.
    bfbeeb is missing meta link: fillable.
    pluwfl is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    bamfsz is missing meta link: fillable.
    lwjdmj is missing meta link: fillable.
    lsyzkh is missing meta link: fillable.
    fwstpx is missing meta link: fillable.
    fqhdne is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    uuypot is missing meta link: fillable.
    tjrbxv is missing meta link: fillable.
    fexqbj is missing meta link: fillable.
    jvdbxh is missing meta link: fillable.
    dajebq is missing meta link: fillable.

  ### (.*) is missing meta link: togglebutton.: 13
    czyfhq is missing meta link: togglebutton.
    pyttso is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    bfbeeb is missing meta link: togglebutton.
    dngvvi is missing meta link: togglebutton.
    igwqpj is missing meta link: togglebutton.
    fexqbj is missing meta link: togglebutton.
    snbvop is missing meta link: togglebutton.
    iqbpie is missing meta link: togglebutton.
    ksecxq is missing meta link: togglebutton.
    xbfgjc is missing meta link: togglebutton.
    zexzrc is missing meta link: togglebutton.
    jqsuky is missing meta link: togglebutton.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 11
    Articulated object fqhdne instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lwjdmj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object emeeke instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lsyzkh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object kfmkbm instances have different scales for base links. This may have broken things during the match links script.
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xbfgjc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object pluwfl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object gpkbiw instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fexqbj instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 4
    window-inmymj-0-left_wing-base_link-P-upper has different pivot offset position (by [ 5.68977587e-01 -1.00148449e-04  1.42420406e-01]). Match pivots on each instance.
    B-bottom_cabinet_no_top-pluwfl-2-link_0-base_link-R-lower has different pivot offset position (by [0.1082358  0.16032175 0.        ]). Match pivots on each instance.
    B-bottom_cabinet_no_top-pluwfl-2-link_1-base_link-P-lower has different pivot offset position (by [0.01808094 0.16032172 0.        ]). Match pivots on each instance.
    window-inmymj-0-right_wing-base_link-P-upper has different pivot offset position (by [ 0.00000000e+00  8.69235298e-05 -1.42415414e-01]). Match pivots on each instance.

  ### (.*) is missing meta link: fluidsink.: 3
    zexzrc is missing meta link: fluidsink.
    czyfhq is missing meta link: fluidsink.
    ksecxq is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 3
    ksecxq is missing meta link: fluidsource.
    czyfhq is missing meta link: fluidsource.
    zexzrc is missing meta link: fluidsource.

  ### (.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.: 2
    inmymj link right_wing has different position in instance 1 compared to instance 0. Position difference: [-0.5        -3.90625    -0.12109375].
    wwhydr link right_wing has different position in instance 2 compared to instance 0. Position difference: [ 0.          1.3125     -0.29296875].

  ### (.*) is missing meta link: heatsource.: 1
    igwqpj is missing meta link: heatsource.


## scenes/Wainscott_0_int
  ### (.*) is missing meta link: fillable.: 22
    dajebq is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    czyfhq is missing meta link: fillable.
    bmsclc is missing meta link: fillable.
    bfbeeb is missing meta link: fillable.
    bamfsz is missing meta link: fillable.
    jvdbxh is missing meta link: fillable.
    qohxjq is missing meta link: fillable.
    fexqbj is missing meta link: fillable.
    jrhgeu is missing meta link: fillable.
    pyttso is missing meta link: fillable.
    zexzrc is missing meta link: fillable.
    pluwfl is missing meta link: fillable.
    jhymlr is missing meta link: fillable.
    owvfik is missing meta link: fillable.
    eobsmt is missing meta link: fillable.
    dngvvi is missing meta link: fillable.
    fwstpx is missing meta link: fillable.
    lwjdmj is missing meta link: fillable.
    dszchb is missing meta link: fillable.
    fqhdne is missing meta link: fillable.
    lsyzkh is missing meta link: fillable.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 14
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fqhdne instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fexqbj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lsyzkh instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xbfgjc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object emeeke instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ktydvs instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object kfmkbm instances have different scales for base links. This may have broken things during the match links script.
    Articulated object gpkbiw instances have different scales for base links. This may have broken things during the match links script.
    Articulated object pluwfl instances have different scales for base links. This may have broken things during the match links script.
    Articulated object qohxjq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lwjdmj instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: togglebutton.: 13
    bfbeeb is missing meta link: togglebutton.
    jqsuky is missing meta link: togglebutton.
    ksecxq is missing meta link: togglebutton.
    snbvop is missing meta link: togglebutton.
    xbfgjc is missing meta link: togglebutton.
    fexqbj is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    zexzrc is missing meta link: togglebutton.
    dngvvi is missing meta link: togglebutton.
    czyfhq is missing meta link: togglebutton.
    igwqpj is missing meta link: togglebutton.
    pyttso is missing meta link: togglebutton.
    iqbpie is missing meta link: togglebutton.

  ### (.*) is missing meta link: fluidsink.: 3
    czyfhq is missing meta link: fluidsink.
    ksecxq is missing meta link: fluidsink.
    zexzrc is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 3
    ksecxq is missing meta link: fluidsource.
    czyfhq is missing meta link: fluidsource.
    zexzrc is missing meta link: fluidsource.

  ### (.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.: 2
    B-bottom_cabinet_no_top-pluwfl-2-link_0-base_link-R-lower has different pivot offset position (by [0.10823512 0.16032354 0.        ]). Match pivots on each instance.
    B-bottom_cabinet_no_top-pluwfl-2-link_1-base_link-P-lower has different pivot offset position (by [0.0180813 0.1603235 0.       ]). Match pivots on each instance.

  ### (.*) is missing meta link: heatsource.: 1
    igwqpj is missing meta link: heatsource.


## scenes/Wainscott_1_int
  ### (.*) is missing meta link: fillable.: 14
    bamfsz is missing meta link: fillable.
    fqhdne is missing meta link: fillable.
    immwzb is missing meta link: fillable.
    ksecxq is missing meta link: fillable.
    omeuop is missing meta link: fillable.
    zotrbg is missing meta link: fillable.
    fdjykf is missing meta link: fillable.
    xsuyua is missing meta link: fillable.
    jvdbxh is missing meta link: fillable.
    nyfebf is missing meta link: fillable.
    qohxjq is missing meta link: fillable.
    xiybkb is missing meta link: fillable.
    nnvyol is missing meta link: fillable.
    jhymlr is missing meta link: fillable.

  ### Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.: 9
    Articulated object ulnafj instances have different scales for base links. This may have broken things during the match links script.
    Articulated object nnvyol instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ohagsq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object lvgliq instances have different scales for base links. This may have broken things during the match links script.
    Articulated object ktydvs instances have different scales for base links. This may have broken things during the match links script.
    Articulated object fqhdne instances have different scales for base links. This may have broken things during the match links script.
    Articulated object jhymlr instances have different scales for base links. This may have broken things during the match links script.
    Articulated object xbfgjc instances have different scales for base links. This may have broken things during the match links script.
    Articulated object bamfsz instances have different scales for base links. This may have broken things during the match links script.

  ### (.*) is missing meta link: togglebutton.: 7
    ksecxq is missing meta link: togglebutton.
    ackppx is missing meta link: togglebutton.
    xsuyua is missing meta link: togglebutton.
    wseglt is missing meta link: togglebutton.
    xbfgjc is missing meta link: togglebutton.
    xiybkb is missing meta link: togglebutton.
    omeuop is missing meta link: togglebutton.

  ### (.*) is missing meta link: fluidsink.: 2
    ksecxq is missing meta link: fluidsink.
    xiybkb is missing meta link: fluidsink.

  ### (.*) is missing meta link: fluidsource.: 2
    ksecxq is missing meta link: fluidsource.
    xiybkb is missing meta link: fluidsource.


In [253]:
from collections import defaultdict
import re

warning_re = re.compile(r"(.*) has too many vertices: (\d+) > 20000")
warn_objs = defaultdict(dict)
for name, x in jsons.items():
    for w in x["warnings"]:
        m = warning_re.fullmatch(w)
        if m:
            warn_objs[name][m.group(1)] = int(m.group(2))
            
print(warn_objs)

defaultdict(<class 'dict'>, {'objects/batch-00': {'L-tag-ohegnp-0': 21794, 'chicken_wire-czoiqw-0': 52305}, 'objects/batch-01': {'L-shoe_rack-spdrzb-0': 59456}, 'objects/batch-02': {'L-radio-vgioak-0': 50123}, 'objects/batch-06': {'zipper-qwsvid-0': 27426}, 'objects/legacy_batch-01': {'swivel_chair-fqbnuk-0-base_link': 36405, 'swivel_chair-lizuow-0-base_link': 25743, 'swivel_chair-lxupys-0-base_link': 35465, 'swivel_chair-mqxrim-0-base_link': 21060, 'swivel_chair-opkoam-0-base_link': 38343}, 'objects/legacy_batch-02': {'table_lamp-pmekkr-0-arm-body-R-lower': 23529, 'table_lamp-pmekkr-0-head-neck-R-lower': 44331, 'table_lamp-pmekkr-0-arm-body-R-upper': 23529, 'table_lamp-pmekkr-0-head-neck-R-upper': 44331, 'table_lamp-pxdhzm-0-head-arm-R-lower': 33474, 'table_lamp-pxdhzm-0-head-arm-R-upper': 33474}, 'objects/legacy_batch-03': {'desk-egzdgz-0-base_link': 20943, 'dishwasher-znfvmj-0-link_0-base_link-P-lower': 31812, 'dishwasher-znfvmj-0-link_1-base_link-P-lower': 31812, 'dishwasher-znfvmj

In [254]:
# Worst offenders
sorted((cnt, obj) for warn_target in warn_objs.values() for obj, cnt in warn_target.items())

[(20082, 'barbecue_sauce_bottle-gfxrnj-0'),
 (20082, 'catsup_bottle-qfvqfm-0'),
 (20082, 'chocolate_sauce_bottle-yegrkf-0'),
 (20082, 'glaze_bottle-zdxagk-0'),
 (20082, 'hot_sauce_bottle-qvpthd-0'),
 (20082, 'mustard_bottle-lgxfyv-0'),
 (20082, 'salsa_bottle-kydilb-0'),
 (20082, 'soy_sauce_bottle-saujjl-0'),
 (20082, 'squeeze_bottle-uzmhdn-0'),
 (20082, 'sugar_syrup_bottle-kdlbbq-0'),
 (20082, 'vanilla_bottle-drevku-0'),
 (20082, 'white_sauce_bottle-gtwngf-0'),
 (20082, 'wine_sauce_bottle-vqtevv-0'),
 (20279, 'pot_plant-kxmvco-0-base_link'),
 (20943, 'desk-egzdgz-0-base_link'),
 (21060, 'swivel_chair-mqxrim-0-base_link'),
 (21794, 'L-tag-ohegnp-0'),
 (22097, 'L-garden_chair-deuron-0'),
 (22097, 'L-garden_chair-deuron-1'),
 (22560, 'room_light-fxtqtn-0'),
 (22560, 'room_light-fxtqtn-1'),
 (22740, 'fridge-jtqazu-0-base_link'),
 (22956, 'L-pedestal_table-ifmhpn-0'),
 (22956, 'L-pedestal_table-ifmhpn-1'),
 (22998, 'C-stockpot-gxiqbw-0'),
 (22998, 'C-stockpot-gxiqbw-1'),
 (23465, 'floor_lam

In [255]:
# Check which of the "articulated object has diff scale" errors are on bad objects
providers = json.loads(pathlib.Path(r"D:\ig_pipeline\artifacts\pipeline\object_inventory.json").read_text())["providers"]
id_providers = {k.split("-")[-1]: v for k, v in providers.items()}
categories = dict([tuple(reversed(k.split("-"))) for k in providers.keys()])
articulation_tmpl = "Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script."
for target, objs in error_matches_by_file[articulation_tmpl].items():
    if not objs:
        continue
    bad_objs = set()
    for match in objs:
        obj = match.group(1)
        if id_providers[obj] != target:
            name = f"{categories[obj]}-{obj}"
            bad_objs.add(name)

    if not bad_objs:
        continue
    
    print(f"{target}: {len(bad_objs)} {bad_objs}")

scenes/Beechwood_0_garden: 6 {'top_cabinet-dmwxyl', 'window-ulnafj', 'bottom_cabinet_no_top-qohxjq', 'bottom_cabinet-immwzb', 'door-lvgliq', 'bottom_cabinet_no_top-spojpj'}
scenes/Beechwood_0_int: 7 {'top_cabinet-dmwxyl', 'window-ulnafj', 'bottom_cabinet_no_top-qohxjq', 'bottom_cabinet-immwzb', 'door-lvgliq', 'door-ohagsq', 'bottom_cabinet_no_top-spojpj'}
scenes/Beechwood_1_int: 9 {'top_cabinet-dmwxyl', 'window-ulnafj', 'bottom_cabinet-kubcdk', 'bottom_cabinet-jhymlr', 'door-lvgliq', 'door-ohagsq', 'window-ithrgo', 'bottom_cabinet_no_top-pluwfl', 'toilet-kfmkbm'}
scenes/Benevolence_0_int: 1 {'door-lvgliq'}
scenes/Benevolence_1_int: 6 {'top_cabinet-dmwxyl', 'top_cabinet-eobsmt', 'window-ulnafj', 'cedar_chest-fwstpx', 'bottom_cabinet_no_top-ufhpbn', 'top_cabinet-lsyzkh'}
scenes/Benevolence_2_int: 2 {'window-ithrgo', 'door-lvgliq'}
scenes/hotel_gym_spa: 2 {'door-nfrbch', 'door-ceddpg'}
scenes/house_single_floor: 1 {'bottom_cabinet_no_top-gjrero'}
scenes/Ihlen_0_int: 3 {'door-ktydvs', 'win

In [257]:
# Get a list of all the objects that show up in the match links relevant errors
mismatched_link_offset_tmpl = "(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*."
objects_for_match_links = error_matches[mismatched_link_offset_tmpl] | error_matches[articulation_tmpl]
print(", ".join(f'"{x.group(1)}"' for x in sorted(objects_for_match_links, key=lambda x: x.group(1))))

"bamfsz", "bamfsz", "bamfsz", "bamfsz", "bmsclc", "ceddpg", "ceddpg", "ceddpg", "dladgw", "dladgw", "dladgw", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dnyzym", "dqnbsj", "dqnbsj", "ehwmol", "ehwmol", "emeeke", "emeeke", "eobsmt", "fexqbj", "fexqbj", "fexqbj", "fqhdne", "fqhdne", "fqhdne", "fwstpx", "fwstpx", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "gemgfz", "ggcyib", "ggcyib", "gjrero", "gpkbiw", "gpkbiw", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "immwzb", "immwzb", "immwzb", "inmymj", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "jhymlr", "jhymlr", "jhymlr", "jrhgeu", "kfmkbm", "kfmkbm", "kfmkbm", "ktydvs", "ktydvs", "ktydvs", "kubcdk", "kwbnhy", "kwbnhy", "kwbnhy", 

In [260]:
import sys
sys.path.append(r"D:\ig_pipeline")
from b1k_pipeline.utils import parse_name

# Get a link of all the objects that are bad cloth
cloth_tmpl = "Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements."
objects_for_cloth = [parse_name(x.group(1)).group('model_id') for x in error_matches[cloth_tmpl]]
print(", ".join(f'"{x}"' for x in sorted(objects_for_cloth)))

"agftpm", "aiftuk", "ajcjzr", "bgvwes", "bpjqwj", "bpjqwj", "bpjqwj", "bpjqwj", "brebdk", "crbvcg", "cuoayr", "dnypbe", "efmfdf", "ekcjci", "eknmzl", "faedft", "feohwy", "fglfga", "ficvss", "fmapwe", "fzldgi", "getwzm", "girtqm", "gtghon", "gxmcpy", "htcikj", "imdhfp", "itrkhr", "ivmsdt", "jfsovh", "jkcugp", "jmujjo", "kclcrj", "kiiium", "klhkgd", "laksie", "ldpbcu", "lqweda", "mksdlu", "mpxtgf", "mxxyva", "njidcf", "nmvvil", "nogevo", "nowqqh", "nzbhjq", "ohvomi", "ohvomi", "ojgwec", "omsbux", "padfds", "pbytey", "pchzob", "prhems", "pvzxyp", "qewdqa", "qofxar", "rcgdde", "remcyk", "rhgweu", "rygqoc", "sehjcp", "shbakk", "shbakk", "stcrad", "sullco", "szdonp", "szdonp", "szdonp", "szdonp", "tfnwti", "thmepr", "tnirgd", "trwest", "ttgike", "uaidmc", "uksirc", "uksirc", "uwzqxm", "uxgjdv", "vfknnl", "vpafgj", "vqbvph", "wbhliu", "wczokr", "xeskbh", "xzuywj", "ydgivr", "ykiiqr", "ylycdz"
